# Setup

In [ ]:
set.seed(1207)

In [ ]:
knitr::opts_chunk$set(echo = TRUE)
knitr::opts_chunk$set(fig.width=20, fig.height=15) 

# Load

"randomcoloR","DECIPHER",'Hmisc',

In [ ]:
#install.packages('PopGenReport',repos='https://mirrors.tongji.edu.cn/CRAN/')
#library(PopGenReport)

In [ ]:
list.of.packages <- c( "tidyverse","Biostrings","ggtree")
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) BiocManager::install(new.packages)
lapply(list.of.packages, library, character.only=T)

In [ ]:
list.of.packages <- c("readr","tidyr","Hmisc","plyr","randomcoloR","PopGenReport") # replace xx and yy with package names
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages)
lapply(list.of.packages, library, character.only=T)

In [ ]:
list.of.packages <- c("ggplot2","ape","cowplot","UpSetR","reshape2","ggpubr","phangorn","factoextra","RColorBrewer","gplots","lsei") # replace xx and yy with package names
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages)
lapply(list.of.packages, library, character.only=T)

In [ ]:
list.of.packages <- c("npsurv","fitdistrplus","scales","maptools","maps","ggthemes","scatterpie","NbClust","radiator","adegenet","rhierbaps","dplyr","dartR","poppr")
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages)
lapply(list.of.packages, library, character.only=T)

In [ ]:
list.of.packages <- c("dendextend","viridis","phylogram","colorspace") # replace xx and yy with package names
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages)
lapply(list.of.packages, library, character.only=T)

In [ ]:
#library(mlplasmids)
library(vcfR)
#library(adegenet)

In [ ]:
color = grDevices::colors()[grep('gr(a|e)y', grDevices::colors(), invert = T)]#设置调色盘

In [ ]:
qual_col_pals = brewer.pal.info[brewer.pal.info$category == 'qual',]
col_vector = unlist(mapply(brewer.pal, qual_col_pals$maxcolors, rownames(qual_col_pals)))#方案二，74种

In [ ]:
output_fig='./output/fig/'
output_file='./output/file/'

In [ ]:
blast<-read.csv('./input/blast.csv',stringsAsFactors=F,header=T,row.names=NULL,na.strings = "")
addblast<-read.csv('./input/addblast.csv',stringsAsFactors=F,header=T,row.names=NULL,na.strings = "")

addout<-left_join(addblast,blast,by='ACC')

write.csv(x =addout, file = './input/addout.csv', row.names = FALSE)

# Processing

## plsdb filtering

In [ ]:
#ndm_ncbi<-read.table('input/ndm_acc.txt',stringsAsFactors=F)

#plsdb<-read.csv('input/plsdb.tsv',sep='\t',header=T,stringsAsFactors=F,row.names=NULL,fill=TRUE,quote="")

#ndm_plsdb<-plsdb[which((plsdb$ACC_NUCCORE %in% unique(ndm_ncbi$V1)) | (plsdb$Identical %in% unique(ndm_ncbi$V1))),]

#fusion plasmids
#fusion = ndm_ncbi[which((duplicated(ndm_ncbi$V1)) & (grepl('NDM',ndm_ncbi$V2))),]$V1

#ndm_plsdb<-ndm_plsdb[-which(ndm_plsdb$ACC_NUCCORE %in% fusion),]

## supplement information 

### mob-suite

### by biopython(reference.title,pmid,journal,abstract,subtype,etc)

In [ ]:
#plsdb_ndm_mob<-read.csv('./input/plsdb_ndm_mob.csv',stringsAsFactors=F,header=T,row.names=NULL,na.strings = "")

#add_gb<-read.csv('./input/add_gb.tsv',sep='\t',stringsAsFactors=F,header=F,row.names=NULL,na.strings = "")
#colnames(add_gb)<-c('ACC_NUCCORE','host','country','isolation.source','reference.title','journal','pmid')

#subtype<-read.csv('./input/ndm592subtype2.txt',sep='\t',stringsAsFactors=F,header=F,row.names=NULL,na.strings = "")
#colnames(subtype)<-c('ACC_NUCCORE','subtype')

#add_ab<-read.csv('./input/ndm_ab.txt',sep='\t',stringsAsFactors=F,header=F,row.names=NULL,quote="",na.strings = "")
#colnames(add_ab)<-c('pmid','abstract')

#plsdb_ndm_mob_gb<-left_join(plsdb_ndm_mob,unique(add_gb),by='ACC_NUCCORE')

#plsdb_ndm_mob_gb_sub<-left_join(plsdb_ndm_mob_gb,unique(subtype),by='ACC_NUCCORE')

#plsdb_ndm_mob_gb_sub_ab<-left_join(plsdb_ndm_mob_gb_sub,unique(add_ab),by='pmid')

In [ ]:
#write.table (plsdb_ndm_mob_gb_sub_ab, file ='./input/complete_plasmid_metadata.txt', sep ="\t", row.names =FALSE, col.names =TRUE, quote =F)

### supplement host

In [ ]:
ndm_all<-read.csv("./input/complete_plasmid_metadata.txt",header=T,stringsAsFactors=F,sep="\t",row.names=NULL)

cols1 <- c('Host_BIOSAMPLE' , 'host')
ndm_all$host_all <- apply( ndm_all[ , cols1 ] , 1 , paste , collapse = "," )

cols2 <- c('IsolationSource_BIOSAMPLE' , 'isolation.source')
ndm_all$isolation_all <- apply( ndm_all[ , cols2 ] , 1 , paste , collapse = "," )

In [ ]:
ndm_all %>%
dplyr::count(host_all)

In [ ]:
other_index<-which(ndm_all$host_all %in% c('Missing,Missing'))
ndm_all$host_all[other_index]<-ndm_all$isolation_all[other_index]

In [ ]:
ndm_all$host_all[which(grepl('water|soil|sediment|sewage|river|wastewater|greenwood|environment', ndm_all$host_all,ignore.case = TRUE))] <- 'Environment'
ndm_all$host_all[which(grepl('hospital|clinical|patient', ndm_all$host_all,ignore.case = TRUE))] <- 'Patient'
ndm_all$host_all[which(grepl('Homo sapiens|Homo|skin|sputum|stool|Transtracheal|rectal|fecal|feces|anus|human|blood|wound|urine|urinary|abdominal|Patient', ndm_all$host_all,ignore.case = TRUE))] <- 'Homo sapiens'
ndm_all$host_all[which(grepl('Canis|cat|chicken|dog|duck|Milvus|pig|goose|milk|swine|bird|dairy|pork|shrimp|retail', ndm_all$host_all,ignore.case = TRUE))] <- 'Animal'
ndm_all$host_all[which(grepl('cucumber', ndm_all$host_all,ignore.case = TRUE))] <- 'Vegetable'

other2_index<-which(!ndm_all$host_all %in% c('Patient','Homo sapiens','Animal','Environment','Vegetable'))
ndm_all$host_all[other2_index]<-'Missing'

In [ ]:
ndm_all %>%
dplyr::count(host_all)

In [ ]:
ndm_all$host_all[which((ndm_all$host_all == 'Missing') & (grepl('Homo sapiens|Homo|skin|sputum|stool|Transtracheal|rectal|fecal|feces|anus|human|blood|wound|urine|urinary|abdominal|Patient|clinical|hospital', ndm_all$reference.title,ignore.case = TRUE)))] <- 'Homo sapiens'
ndm_all$host_all[which((ndm_all$host_all == 'Missing') & (grepl('Homo sapiens|Homo|skin|sputum|stool|Transtracheal|rectal|fecal|feces|anus|human|blood|wound|urine|urinary|abdominal|Patient|clinical|hospital', ndm_all$abstract,ignore.case = TRUE)))] <- 'Homo sapiens'

In [ ]:
ndm_all$host_all[which((ndm_all$host_all == 'Missing') & (grepl('farm|goose|agricultural|animal|pigs', ndm_all$reference.title,ignore.case = TRUE)))] <- 'Animal'
ndm_all$host_all[which((ndm_all$host_all == 'Missing') & (grepl('farm|goose|agricultural|animal|pigs|Canis|cat|chicken|dog|duck|Milvus|pig|goose|milk|swine|bird|dairy|pork|shrimp|retail|food', ndm_all$abstract,ignore.case = TRUE)))] <- 'Animal'

In [ ]:
ndm_all$host_all[which((ndm_all$host_all == 'Missing') & (grepl('environment', ndm_all$reference.title,ignore.case = TRUE)))] <- 'Environment'
ndm_all$host_all[which((ndm_all$host_all == 'Missing') & (grepl('water|soil|sediment|sewage|river|wastewater|greenwood|environment', ndm_all$abstract,ignore.case = TRUE)))] <- 'Environment'

In [ ]:
ndm_all %>%
dplyr::count(host_all)

In [ ]:
#by manually searching
ndm_all[which( (grepl('Missing', ndm_all$host_all,ignore.case = TRUE))
              & !(ndm_all$reference.title == 'Direct Submission')
              & !(ndm_all$journal == 'Unpublished')),]$host_all<-rep('Homo sapiens',6)

### supplement subtype

In [ ]:
ndm_all$subtype[which((ndm_all$subtype == 'Missing') & (grepl('NDM-1', ndm_all$reference.title,ignore.case = TRUE)))] <- 'NDM-1'
ndm_all$subtype[which((ndm_all$subtype == 'Missing') & (grepl('NDM-5', ndm_all$reference.title,ignore.case = TRUE)))] <- 'NDM-5'
ndm_all$subtype[which((ndm_all$subtype == 'Missing') & (grepl('NDM-9', ndm_all$reference.title,ignore.case = TRUE)))] <- 'NDM-9'

ndm_all$subtype[which((ndm_all$subtype == 'Missing') & (grepl('NDM-1', ndm_all$abstract,ignore.case = TRUE)))] <- 'NDM-1'
ndm_all$subtype[which((ndm_all$subtype == 'Missing') & (grepl('NDM-5', ndm_all$abstract,ignore.case = TRUE)))] <- 'NDM-5'

In [ ]:
ndm_all[which(ndm_all$subtype == 'Missing'),]$ACC_NUCCORE

In [ ]:
#by manually searching
ndm_all[which(ndm_all$subtype == 'Missing'),]$subtype<-c('NDM-5','NDM-1','NDM-1')

In [ ]:
ndm_all %>%
dplyr::count(subtype)

### supplement location

In [ ]:
cols3 <- c('country' , 'loc_parsed','Location_BIOSAMPLE')
ndm_all$location_all <- apply( ndm_all[ , cols3 ] , 1 , paste , collapse = ";" )

In [ ]:
ndm_all$location_all[which(grepl('China|Taiwan|Hong|Macao', ndm_all$location_all,ignore.case = TRUE))] <- 'China'

ndm_all$location_all[which(grepl('United Kingdom', ndm_all$location_all,ignore.case = TRUE))] <- 'United Kingdom'

ndm_all$location_all[which(grepl('South Africa', ndm_all$location_all,ignore.case = TRUE))] <- 'South Africa'

ndm_all$location_all[which(grepl('Korea', ndm_all$location_all,ignore.case = TRUE))] <- 'South Korea'

ndm_all$location_all[which(grepl('Democratic', ndm_all$location_all,ignore.case = TRUE))] <- 'Denmark'

ndm_all$location_all[which(grepl('USA|America', ndm_all$location_all,ignore.case = TRUE))] <- 'United States'

In [ ]:
ndm_all$location_all[which(grepl('missing', ndm_all$location_all,ignore.case = TRUE))]<-ndm_all$Location_BIOSAMPLE[which(grepl('missing', ndm_all$location_all,ignore.case = TRUE))]

In [ ]:
ndm_all$location_all <- str_split_fixed(ndm_all$location_all, "\\;", 2)[,1]

In [ ]:
ndm_all$location_all <- str_split_fixed(ndm_all$location_all, "\\:", 2)[,1]

In [ ]:
ndm_all %>% 
    dplyr::count(location_all)

In [ ]:
ndm_all$location_all[which(grepl('China|Taiwan|Hong|Macao|Lishui|beijing', ndm_all$abstract,ignore.case = TRUE))] <- 'China'
ndm_all$location_all[which(grepl('China|Taiwan|Hong|Macao|Lishui|beijing', ndm_all$reference.title,ignore.case = TRUE))] <- 'China'

In [ ]:
all_country=c('Italy','Czech','United Arab Emirates','South Korea','Myanmar','Latin American','Poland',
             'Portugal','United Kingdom','Bulgaria','Thailand','Canada','India','Singapore','Romania',
             'United States','Brazil','German','Lebanese','Argentina','Denmark','Japan','South Africa',
             'Morocco','Afghanistan')
for (x in all_country){
    ndm_all$location_all[which(grepl(x, ndm_all$abstract,ignore.case = TRUE))] <- x
    ndm_all$location_all[which(grepl(x, ndm_all$reference.title,ignore.case = TRUE))] <- x
}

In [ ]:
ndm_all$location_all[which(grepl('Lebanese', ndm_all$location_all,ignore.case = TRUE))] <- 'Lebanon'
ndm_all$location_all[which(grepl('German', ndm_all$location_all,ignore.case = TRUE))] <- 'Germany'

In [ ]:
ndm_all %>% 
    dplyr::count(location_all)

In [ ]:
ndm_all[which( (grepl('Missing', ndm_all$location_all,ignore.case = TRUE))
              & !(ndm_all$reference.title == 'Direct Submission')
              & !(ndm_all$journal == 'Unpublished')),]$location_all<-c('China','Czech','Czech','Australia','Australia','Australia','United States','China',
                                                                     'China','India','India','Japan','China','Italy','Czech','China','China','France',
                                                                      'Japan','France') 

In [ ]:
ndm_all$Location2<-ndm_all$location_all

In [ ]:
ndm_all$Location2[which(grepl('Afghanistan|Armenia|Azerbaijan|Bahrain|Viet|Bangladesh|Bhutan|Brunei|Cambodia|China|Cyprus|East Timor|Georgia|India|Indonesia|Iran|Iraq|Israel|Japan|Jordan|Kazakhstan|Kuwait|Kyrgyzstan|Laos|Lebanon|Malaysia|Maldives|Mongolia|Myanmar|Nepal|North Korea|Oman|Pakistan|State of Palestine|Jerusalem|Philippines|Qatar|Russia|Saudi Arabia|Singapore|South Korea|Sri Lanka|Syria|Taiwan|Tajikistan|Thailand|Turkey|Turkmenistan|United Arab|Uzbekistan|Vietnam|Yemen|Lebanese', ndm_all$location_all,ignore.case = TRUE))] <- 'Asia'
ndm_all$Location2[which(grepl('Albania|Andorra|Armenia|Austria|Azerbaijan|Belarus|Belgium|Bosnia and Herzegovina|Bulgaria|Croatia|Cyprus|Czech|Denmark|Estonia|Finland|France|Georgia|Germany|Greece|Hungary|Iceland|Republic of Ireland|Italy|Kazakhstan|Latvia|Liechtenstein|Lithuania|Luxembourg|Malta|Moldova|Monaco|Montenegro|Netherlands|North Macedonia|Norway|Poland|Portugal|Romania|Russia|San Marino|Serbia|Slovakia|Slovenia|Spain|Sweden|Switzerland|Turkey|Ukraine|United Kingdom|Vatican City', ndm_all$location_all,ignore.case = TRUE))] <- 'Europe'
ndm_all$Location2[which(grepl('Nigeria|South Africa|Egypt|Algeria|Angola|Morocco|Kenya|Ethiopia|Libya|Ghana|Malawi|Niger|Tanzania', ndm_all$location_all,ignore.case = TRUE))] <- 'Africa'
ndm_all$Location2[which(grepl('Anguilla|Antigua and Barbuda|Aruba|Bahamas|Barbados|Belize|Bermuda|Bonaire|British Virgin Islands|Canada|Cayman Islands|France|Costa Rica|Cuba|Curaçao|Dominica|Dominican Republic|El Salvador|Venezuela|Greenland|Grenada|France|Guantanamo Bay Naval Base|Guatemala|Haiti|Honduras|Jamaica|France|Mexico|Montserrat|Brades|Nicaragua|Nueva Esparta|Panama|Puerto Rico|Saba|San Andrés and Providencia|Saint Barthélemy|Saint Kitts and Nevis|Saint Lucia|Saint Martin|Saint Pierre and Miquelon|Saint Vincent and the Grenadines|Sint Eustatius|Sint Maarten|Trinidad and Tobago|Turks and Caicos Islands|United States|United States Virgin Islands|USA', ndm_all$location_all,ignore.case = TRUE))] <- 'North America'
ndm_all$Location2[which(grepl('Argentina|Bolivia|Sucre|Norway|Brazil|Chile|Colombia|Ecuador|Falkland Islands|French Guiana|Guyana|Paraguay|Peru|South Georgia and the South Sandwich Islands|Suriname|Uruguay|Venezuela', ndm_all$location_all,ignore.case = TRUE))] <- 'South America'
ndm_all$Location2[which(grepl('Australia|Fiji|Kiribati|Marshall Islands|Majuro|Micronesia|Nauru|New Zealand|Palau|Papua New Guinea|Samoa|Solomon Islands|Tonga|Tuvalu|Vanuatu|latin', ndm_all$location_all,ignore.case = TRUE))] <- 'Australia'

In [ ]:
ndm_all %>%
    dplyr::count(Location2)

# Length Distribution

## by host

In [ ]:
ndm_all_del_missing<-ndm_all[which((ndm_all$subtype %in% c('NDM-1','NDM-5','NDM-9')) & !(ndm_all$host_all %in% c('Vegetable','Missing'))),]

ndm_all_del_missing<-ndm_all_del_missing[-which(ndm_all_del_missing$subtype %in% c('NDM-4','NDM-6')),]
#ndm_all_del_missing[which(ndm_all_del_missing$subtype %in% c('NDM-4','NDM-6')),]$subtype<-'Others'

#ndm_all_del_missing$subtype <- factor(ndm_all_del_missing$subtype, levels=c("Others", "NDM-9", "NDM-5","NDM-1"), ordered=TRUE)

In [ ]:
source('./geom_flat_violin.R')

In [ ]:
g1 <- ggplot(data = ndm_all_del_missing, aes(y = Length_NUCCORE/1e3, x = subtype, fill = subtype)) +
geom_flat_violin(position = position_nudge(x = .2, y = 0), alpha = .8) +
geom_point(aes(y = Length_NUCCORE/1e3, color = subtype), position = position_jitter(width = .35), size = .5, alpha = 0.8) +
geom_boxplot(width = .1, guides = FALSE, outlier.shape = NA, alpha = 0.5) +
facet_wrap(~host_all)+
expand_limits(x =3) +
guides(fill = FALSE) +
guides(color = FALSE) +
scale_color_brewer(palette = "Set1") +
scale_fill_brewer(palette = "Set1") +
labs(x='NDM Subtype',y='Plasmid Length (kbp)', title = '')+
coord_flip() +
theme_bw()

g1

## by species

In [ ]:
ndm_all_del_missing2<-ndm_all[which((!(ndm_all$taxon_genus_name %in% "Missing")) & (!(ndm_all$host_all %in% c('Vegetable','Missing')))),]

In [ ]:
colnames(ndm_all_del_missing2)

In [ ]:
ndm_all %>%
dplyr::count(taxon_genus_name)

In [ ]:
ndm_all_del_missing3<-ndm_all_del_missing2[which(ndm_all_del_missing2$taxon_genus_name %in% c('Escherichia','Klebsiella','Acinetobacter','Salmonella')),]

ndm_all_del_missing2[which(ndm_all_del_missing2$taxon_family_name %in% c('Vibrionaceae','Yersiniaceae','Morganellaceae')),]$taxon_family_name<-'Others'

In [ ]:
g2 <- ggplot(data = ndm_all_del_missing3, aes(y = Length_NUCCORE/1e3, x = taxon_genus_name, fill = taxon_genus_name)) +
geom_flat_violin(position = position_nudge(x = .2, y = 0), alpha = .8) +
geom_point(aes(y = Length_NUCCORE/1e3, color = taxon_genus_name), position = position_jitter(width = .35), size = .5, alpha = 0.8) +
geom_boxplot(width = .1, guides = FALSE, outlier.shape = NA, alpha = 0.5) +
facet_wrap(~host_all)+
expand_limits(x = 5.25) +
guides(fill = FALSE) +
guides(color = FALSE) +
scale_color_brewer(palette = "Dark2") +
scale_fill_brewer(palette = "Dark2") +
labs(x='Bacteria Genus',y='Plasmid Length (kbp)', title = '')+
coord_flip() +
theme_bw()

g2

In [ ]:
#options(repr.plot.width = 13, repr.plot.height = 5)
byhost_2<-plot_grid(g1,g2,rel_widths = c(1, 1.1),scale=0.95)
byhost_2
#ggsave(paste0(output_fig, "violin_boxplot_col2.pdf", sep=""), byhost_2,width = 12, height = 4)

## stats

In [ ]:
ndm_all$n<-rep(1,nrow(ndm_all))

FindMode <- function(x) {

    ux <- unique(x)

    ux[which.max(tabulate(match(x, ux)))]

}

ndm_all %>%
dplyr::group_by(subtype) %>%
dplyr::summarize(total_pl=sum(n),
            mean_pl = mean(Length_NUCCORE),
            median_pl = median(Length_NUCCORE),
            mode_pl=FindMode(Length_NUCCORE),
            max_pl=max(Length_NUCCORE),
            min_pl=min(Length_NUCCORE))

In [ ]:
sample<-ndm_all %>%
    dplyr::count(ACC_BIOSAMPLE)

sample<-sample[-1,]

colnames(sample)[2]<-"pl_number"

sample %>% 
dplyr::count(pl_number)

# Plasmid Annotation

- Knowledge of the relationships between plasmid features and host taxonomy is important in order to understand how the plasmids have been spread among microbes. 

- plasmids in the GenBank database into six mobility (MOB) types (MOBC, MOBF, MOBH, MOBP, MOBQ, and MOBV) according to the amino acid sequences of their relaxase proteins. An additional classification was performed based on the plasmids' T4SS involved in mating pair formation (MPF) during conjugation. Smillie et al. proposed four classes of MPF (MPFF, MPFG, MPFI, and MPFT) according to the T4SS amino acid sequences 

- Incompatibility (Inc) is defined as the inability of plasmids sharing similar replication and partition systems to be propagated stably in the same host cell line. Inc groups have been independently classified in three different genera; there are 27 Inc groups in Enterobacteriaceae, 14 Inc groups of Pseudomonas, and approximately 18 Inc groups in Staphylococcus.
- Classification into an Inc group is always based on the amino acid sequence of the replication initiation (Rep) protein (replicon typing)

## Rip 

In [ ]:
ndm_all$rep <- str_split_fixed(ndm_all$plasmidfinder, "\\_", 2)[,1]

ndm_all$rep2<-ndm_all$rep

ndm_all$rep2 <- gsub(pattern = 'IncI2\\(Delta\\)',replacement = 'IncI2', ndm_all$rep2)
ndm_all$rep2 <- gsub(pattern = 'IncI\\(Gamma\\)',replacement = 'IncI complex', ndm_all$rep2)
ndm_all$rep2 <- gsub(pattern = '^IncI$',replacement = 'IncI complex', ndm_all$rep2)

#ndm_all$rep2[which(grepl('Col', ndm_all$rep))] <- 'Col-Like'

ndm_all$rep2[which(grepl('FIB', ndm_all$rep))] <- 'IncFIB'
 
ndm_all$rep2[which(grepl('FIA', ndm_all$rep))] <- 'IncFIA'

ndm_all$rep2[which(grepl('FII', ndm_all$rep))] <- 'IncFII'

ndm_all$rep2[which(grepl('p0111', ndm_all$rep))] <- 'IncFII'

ndm_all$rep2[which(grepl('HI1B', ndm_all$rep))] <- 'IncHI1B'

ndm_all$rep2[which(grepl('IncA|IncC', ndm_all$rep))] <- 'IncA/C'

ndm_all$rep2[which(grepl('IncL|IncM', ndm_all$rep))] <- 'IncL/M'

ndm_all$rep2[which(grepl('HI1', ndm_all$rep))] <- 'IncHI1'

ndm_all$rep2[which(grepl('HI2', ndm_all$rep))] <- 'IncHI2'

ndm_all$rep2[which(grepl('IncI1|IncB/O/K/Z|IncI2', ndm_all$rep))] <- 'IncI complex'

ndm_all$rep2[which(grepl('X1|X4|X6', ndm_all$rep))] <- 'IncX1/X4/X6'

#ndm_all$rep2[which(grepl('IncT|IncW|IncY', ndm_all$rep))] <- 'IncT/W/Y'

#ndm_all$rep2[which(grepl('^$', ndm_all$rep))] <- 'Unknown'

In [ ]:
ndm_all %>%
    dplyr::count(rep)

In [ ]:
ndm_all %>%
    dplyr::count(rep2)

## Relaxases

In [ ]:
ndm_all$mob <- str_split_fixed(ndm_all$relaxase_type, "\\,", 2)[,1]

In [ ]:
ndm_all$mob[which(grepl('^$|-', ndm_all$mob))] <- 'Missing'

In [ ]:
ndm_all$mob <- gsub(pattern = 'MOB',replacement = 'MOB_', ndm_all$mob)

In [ ]:
ndm_all %>%
    dplyr::count(mob)

In [ ]:
ndm_all$mpf <- ndm_all$mpf_type

In [ ]:
ndm_all$mpf[which(grepl('-', ndm_all$mpf))] <- 'Missing'
ndm_all$mpf[which(is.na(ndm_all$mpf))]<-'Missing'

In [ ]:
ndm_all %>%
    dplyr::count(mpf)

## Visualizing all Rip and rel combinations¶

In [ ]:
#ndm_all$create_date<-str_split_fixed(ndm_all$CreateDate_NUCCORE, "\\/", 2)[,1]
ndm_all$create_date<-ndm_all$CreateDate_NUCCORE
ndm_all$create_date

In [ ]:
ndm_all_del_missing3<-ndm_all[which( (ndm_all$rep2 != 'Missing') & (ndm_all$mob != 'Missing') & (ndm_all$mpf != 'Missing') ),]

In [ ]:
pl_rep_features <- subset(ndm_all_del_missing3, select = c("ACC_NUCCORE","rep2","Length_NUCCORE","create_date","subtype","host_all"))
colnames(pl_rep_features)[2]<-"annotation"
pl_mob_features <- subset(ndm_all_del_missing3, select = c("ACC_NUCCORE","mob","Length_NUCCORE","create_date","subtype","host_all"))
colnames(pl_mob_features)[2]<-"annotation"
pl_mpf_features <- subset(ndm_all_del_missing3, select = c("ACC_NUCCORE","mpf","Length_NUCCORE","create_date","subtype","host_all"))
colnames(pl_mpf_features)[2]<-"annotation"
pl_features_length<-rbind(pl_rep_features,pl_mob_features,pl_mpf_features)
pl_features_length <- pl_features_length[! duplicated(pl_features_length),]

In [ ]:
pl_features<-pl_features_length[,c(1,2)]
pl_features

In [ ]:
unique_values_pl <- unique(pl_features)
upset_pl_features <- dcast(unique_values_pl, formula = ACC_NUCCORE ~ annotation, fun.aggregate = length)
rownames(upset_pl_features)<-upset_pl_features[,1] #将数据框的第一列作为行名
upset_pl_features<-upset_pl_features[,-1] #将数据框的第一列删除，只留下剩余的列作为数据
upset_pl_features<-upset_pl_features[-which(!duplicated(upset_pl_features)),]
upset_pl_features<-upset_pl_features[,which(colSums(upset_pl_features) > 0)]
upset_pl_features
#dcast，formula左行标签，右列标签，fun.aggregate填充。acast没有行标签，可直接计算

In [ ]:
upset_pl_features$ACC_NUCCORE <- rownames(upset_pl_features)
rownames(upset_pl_features)<-NULL

In [ ]:
pl_date<-pl_features_length[,c(1,4)]
unique_date_pl <- unique(pl_date)
upset_pl_features<-left_join(upset_pl_features,unique_date_pl,by='ACC_NUCCORE')

In [ ]:
pl_subtype<-pl_features_length[,c(1,5)]
unique_subtype_pl <- unique(pl_subtype)
upset_pl_features<-left_join(upset_pl_features,unique_subtype_pl,by='ACC_NUCCORE')
upset_pl_features

In [ ]:
pl_host<-pl_features_length[,c(1,6)]
unique_host_pl <- unique(pl_host)
upset_pl_features<-left_join(upset_pl_features,unique_host_pl,by='ACC_NUCCORE')
upset_pl_features

upset_pl_features<-upset_pl_features[,-29]

In [ ]:
upset_pl_features$create_date

In [ ]:
upset_pl_features$create_date1 <- as.Date( upset_pl_features$create_date, '%Y/%m/%d')
#upset_pl_features$create_date1 <- as.integer(upset_pl_features$create_date1 - as.Date("2011/07/09"))
upset_pl_features$create_date2<-str_split_fixed(upset_pl_features$create_date, "\\/", 2)[,1]

In [ ]:
upset_pl_features$create_date1

In [ ]:
upset_pl_features$create_date2<-as.numeric(upset_pl_features$create_date2)

In [ ]:
pl_length <- pl_features_length[,c(2,3)]
rep_mean_pl<-pl_length %>%
  dplyr::group_by(annotation) %>%
  dplyr::summarize(Average_plasmid_length = mean(Length_NUCCORE)/1e3)
rep_mean_pl

<https://github.com/hms-dbmi/UpSetR/blob/master/R/scatter_plot.R>

<https://github.com/hms-dbmi/UpSetR>

In [ ]:
scatter_plot <- function(mydata, x, y){
    mydata<-mydata[which((mydata$subtype %in% c('NDM-1','NDM-4','NDM-5','NDM-6','NDM-7','NDM-9'))),]
  att_plot <-(ggplot(data = mydata, aes_string(x = x, y = y, colour = "color"))
               + geom_point(shape=16,position=position_jitter(width = 0.5, height = 0.35),size=0.5) 
               + scale_color_identity()
               #+ylab("subtype of ndm")+xlab("craete date")
               +scale_x_date(name="Date",breaks=date_breaks("2 year"),labels = date_format("%Y"),limits=as.Date(c("2011-07-29","2020-11-17")))
               +ylab("NDM Subtype")
               + theme(panel.background = element_rect(fill = "white"),
                       plot.title = element_text(vjust = 1.3),
                       panel.grid.minor = element_blank(),
                       panel.grid.major = element_blank(),
                       #axis.text.x=element_text(angle=90, hjust=1)#选择x轴标签
                       axis.title.y = element_text(vjust = 1.3, size = 8.3),
                       axis.title.x = element_text(size = 8.3),
                       axis.line.x=element_line(colour="black"),#优先级高于axis.ticks
                       axis.line.y=element_line(colour="black"),#同上
                       plot.margin=unit(c(0.5,0.2,0.2,1), "cm"))# margin(t, r, l, b)
               #+coord_flip()
)
}

In [ ]:
bar_plot <- function(mydata, x){
  att_plot <- (ggplot(data = mydata, aes_string(x = x, fill = "color")) 
               + scale_fill_identity()
               + geom_histogram(stat='count'
                         #,position='dodge'
                         ,binwidth = 1
                         )
               +scale_x_continuous(breaks = c(2011,2013,2015,2017,2019),name="Date")
               +scale_y_continuous(name="Freqency",expand=c(0,0))
               + theme(panel.background = element_rect(fill = "white"),
                       plot.title = element_text(vjust = 1.5),
                       panel.border = element_blank(),
                       panel.grid.minor = element_blank(),
                       panel.grid.major = element_blank(),
                       axis.title.y = element_text(vjust = 1.3, size = 8.3),
                       axis.line.x=element_line(colour="black"),#优先级高于axis.ticks
                       axis.line.y=element_line(colour="black"),#同上
                       axis.title.x = element_text(size = 8.3),
                       plot.margin=unit(c(0.5,0.2,0.2,1), "cm"))
             +coord_flip()
)
}

In [ ]:
colnames(upset_pl_features)

In [ ]:
#col<-sample(col_vector,5)
#barplot(rep(1,5),col=col)
#col

In [ ]:
#pdf("./output/fig/plasmid_annotation.pdf")

upset(upset_pl_features,
      sets = c("IncA/C","IncFIA","IncFIB","IncFII","IncHI1","IncHI2","IncI complex","IncL/M","IncN","IncN2",
        "IncQ1","IncR","IncX1/X4/X6","IncX3","MOB_F","MOB_H","MOB_P","MPF_F","MPF_I","MPF_T"),     
      #nintersects = 1000,
      nsets = 21, 
      number.angles = 2, 
      point.size = 1.5 , 
      line.size = 0.3, 
      mainbar.y.label = "Number of plasmids per combination", sets.x.label = "Number of plasmids", 
      order.by = "freq",
      keep.order = TRUE,
      #matrix.color = 'coral2',
      main.bar.color = 'black',
      text.scale = c(1.3, 1.3, 1, 1, 0.6, 1), #六个数字，分别控制c(intersection size title, intersection size tick labels, set size title, set size tick labels, set names, numbers above bars)
      
    queries = list(list(query = intersects,   #UpSetR 内置的intersects query
    params = list("MPF_T","MOB_P","IncX3"), ##指定作用的交集
    #color = '#E7298A', ##设置颜色，未设置会调用默认调色板
    active = F),   # TRUE：条形图被颜色覆盖，FALSE：条形图顶端显示三角形
    #query.name = "Drama"), # 添加query图例
    list(query = intersects,   #UpSetR 内置的intersects query
    params = list("MPF_F","MOB_F","IncFII"), ##指定作用的交集
    #color = '#FF7F00', ##设置颜色，未设置会调用默认调色板
    active = F),
    list(query = intersects,   #UpSetR 内置的intersects query
    params = list("MPF_F","MOB_H","IncA/C"), ##指定作用的交集
    #color = '#B3DE69', ##设置颜色，未设置会调用默认调色板
    active = F),
    list(query = intersects,   #UpSetR 内置的intersects query
    params = list("MPF_F","MOB_F","IncFIB"), ##指定作用的交集
    #color = '#F1E2CC', ##设置颜色，未设置会调用默认调色板
    active = F)),

     set.metadata = list(data = rep_mean_pl, plots = list(list(type = "hist", 
     column = "Average_plasmid_length", assign = 27))),
    
    attribute.plots = list(gridrows = 32, #添加属性图
    plots = list(
    list(plot = scatter_plot, 
                             x = "create_date1", y = "subtype",#横纵轴的变量
                            #x = "create_date1", y = "host_all",
                            queries = T), #T 则显示出上面queries定义的颜色
   list(plot = bar_plot, x = "create_date2", queries = T)), ncols = 2) # 添加的图分两列
  # list(plot = scatter_plot, x = "create_date1", y = "host_all",queries = T)), ncols = 2)
   #boxplot.summary = c("create_date", "host_all")
   #query.legend = "top"#query图例在最上方 
      
) 

#dev.off()

## Stats

In [ ]:
rep_per_subtype <- ndm_all %>%
  group_by(subtype) %>%
  dplyr::count(rep2)
rep_per_subtype

In [ ]:
rep_per_mob <- ndm_all %>%
  group_by(rep2) %>%
  dplyr::count(mob)
rep_per_mob

## Inc map

In [ ]:
pl_map_data<-ndm_all[,c("ACC_NUCCORE","location_all","rep2","subtype")]
pl_map_data<-pl_map_data[!grepl("^$",pl_map_data$location_all),]
pl_map_data<-pl_map_data[-which(pl_map_data$rep2=="Missing"),]
pl_map_data<-pl_map_data[-which(pl_map_data$location_all=="Missing"),]
pl_map_data<-unique(pl_map_data)
cols4 <- c('rep2' , 'subtype')
pl_map_data$combination1 <- apply( pl_map_data[ , cols4 ] , 1 , paste , collapse = ";" )
pl_map_data

In [ ]:
worldmap_pl_data<- dcast(pl_map_data, formula = location_all ~ rep2, fun.aggregate = length)
worldmap_pl_data

In [ ]:
#write.csv(x =worldmap_pl_data, file = './input/worldmap_pl_data.csv', row.names = FALSE)

In [ ]:
geo_df<-read.csv("./input/worldmap_pl_data_add.csv",header=T,stringsAsFactors=F,sep=",",row.names=NULL)

In [ ]:
geo_df

In [ ]:
worldmap = map_data("world")

In [ ]:
fig1 = ggplot(geo_df, aes(long, lat)) +
geom_polygon(data = worldmap, aes(x = long, y = lat,group=group, fill = NA),fill = 'white',color = "grey70", size = 0.25)+
geom_scatterpie(data=geo_df,aes(x=longitude, y=latitude, group=location_all,r=radius), cols=LETTERS[1:16],size=0.1,alpha=.5) +
geom_scatterpie_legend(geo_df$radius, x=-160, y=-55, n=4, labeller=function(x) x^2)+
scale_fill_manual(values = sample(col_vector, 16),name="Inc Type",breaks = c('A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P'), 
                    labels = c("IncA/C","IncFIA","IncFIB","IncFII","IncHI1","IncHI2","IncI complex","IncL/M","IncN","IncN2",
        "IncQ1","IncR","IncT","IncW","IncX1/X4/X6","IncX3"))+
#scale_fill_discrete(name="Inc Type",breaks = c('A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q'), 
                    #labels = c('Col.like','IncA/C','IncB/O/K/Z','IncFIA','IncFIB','IncFII','IncHI1','IncHI2','IncI','IncL/M','IncN','IncN2','IncQ1','IncR','IncT/W/Y','IncX1/X4/X6','IncX3'))+
#coord_equal()+
#coord_cartesian()+
#scale_y_continuous(breaks = (-3:3)*30)+
#cale_x_continuous(breaks = (-6:6)*30)+
#labs(x="Longitude", y="Latitude") +
theme(panel.grid.major =element_blank(), 
      panel.grid.minor = element_blank(),
      #panel.background =element_blank(),
      plot.background = element_rect(fill = '#C3E2F9'),
      panel.background = element_rect(fill = '#C3E2F9'),
      axis.title.x=element_blank(),
      axis.text.x=element_blank(),
      axis.ticks.x=element_blank(),
      axis.title.y=element_blank(),
      axis.text.y=element_blank(),
      axis.ticks.y=element_blank())  +
theme(legend.position="top",
      legend.background = element_rect(fill = '#C3E2F9'),
      legend.text = element_text(size = 7),
      legend.title = element_text(size = 10,face = 'bold'),
      legend.key.size=unit(0.3,'cm'))

##ggsave(paste0(output_dir, "mcr-1_worldmap.tiff", sep=""), fig1, width = 9, height = 5)
##ggsave(paste0(output_fig, "ndmplamids_Inc_worldmap.pdf", sep=""), fig1, width = 9, height = 5)
fig1

write.csv(x =ndm_all, file = '/root/jupyter_projects/ndm_plasmid_2021/input/ndm_all_processed.csv', row.names = FALSE)

#  Inter pop

## Mash Distance

In [ ]:
ndm_all_mash<-read.table("./input/mash/ndm_all2all_mash_e_incx3.1.tab",header=F,stringsAsFactors=F,sep="\t",row.names=NULL)

In [ ]:
colnames(ndm_all_mash)<-c("ACC_NUCCORE","query_id","mash_distance","p_value","matching_hashes")

parsing_mash<-ndm_all_mash %>% dplyr::select(ACC_NUCCORE,mash_distance)
parsing_mash$ACC_NUCCORE <- gsub(pattern = '\\.[0-9]\\_.*fasta',replacement = '', parsing_mash$ACC_NUCCORE)
parsing_mash$ACC_NUCCORE<-toupper(parsing_mash$ACC_NUCCORE)
#parsing_mash<-arrange(parsing_mash,mash_distance)

pl_concatenated <- parsing_mash[,1]


len_strains <- length(unique(pl_concatenated))

mash_df <- NULL
first_strain <- as.character(parsing_mash[1,1])
#循环行
for(i in 1:length(pl_concatenated))
{
  if(parsing_mash[i,1]== first_strain)
  {
    first_index <- i
    second_index <- first_index + len_strains
    df_selection <- parsing_mash[c(first_index:second_index),2]
    mash_df <- rbind(mash_df, df_selection)
  }
}
mash_df <- mash_df[,c(1:len_strains)]
sum(is.infinite(mash_df))

In [ ]:
head(parsing_mash)

In [ ]:
dim(mash_df)

In [ ]:
ACCS<-parsing_mash$ACC_NUCCORE[1:436]

ndm_all2one_mash<-read.table("./input/complete_fasta/ndm_all_mash.tab",header=F,stringsAsFactors=F,sep="\t",row.names=NULL)
colnames(ndm_all2one_mash)<-c("ACC_NUCCORE","query_id","mash_distance","p_value","matching_hashes")
ndm_all2one_mash$ACC_NUCCORE <- gsub(pattern = '\\.[0-9]\\.fasta',replacement = '', ndm_all2one_mash$ACC_NUCCORE)

ndm_all2one_mash$ACC_NUCCORE<-toupper(ndm_all2one_mash$ACC_NUCCORE)

ACCS<-ndm_all2one_mash$ACC_NUCCORE

In [ ]:
colnames(mash_df)<-ACCS
rownames(mash_df)<-ACCS

head(mash_df)

## Tree

In [ ]:
hc_mash<-hclust(as.dist(mash_df),method="ward.D2",members=NULL)
tree_hc_mash<-as.phylo(hc_mash)

hc_mash

tree_hc_mash

colnames(ndm_all_mash_info)

ndm_all_mash_info<-left_join(ndm_all2one_mash,unique(ndm_all),by="ACC_NUCCORE")
#ndm_all_mash_info

tree_strains_info <- data.frame(label = as.character(ndm_all_mash_info$ACC_NUCCORE),
           host = as.character(ndm_all_mash_info$host_all))
tree_strains_info %>%
dplyr::count(host)

tree_country_info <- data.frame(label = as.character(ndm_all_mash_info$ACC_NUCCORE),
           country = as.character(ndm_all_mash_info$Location2))
tree_country_info %>%
dplyr::count(country)

tree_strains_info_del<-tree_strains_info[-which(tree_strains_info$host %in% c('Missing','Vegetable')),]
plot_isolation <- facet_plot(ggtree(tree_hc_mash, layout = 'rectangular'), panel = 'Source', data = tree_strains_info_del, geom=geom_tile, aes(x=as.numeric(as.factor(host)), fill = host)) + theme_tree2(legend.position = 'bottom')   

tree_country_info_del<-tree_country_info[-which(tree_country_info$country %in% c('Missing')),]
plot_isolation2<- facet_plot(plot_isolation, panel = 'Continent' ,data = tree_country_info_del, geom=geom_tile, aes(x=as.numeric(as.factor(country)), fill = country)) + theme_tree2(legend.position = 'bottom')  
#ggtree(tree_hc_mash, layout = 'rectangular')

plot_isolation2

In [ ]:
# pupolation clustering also performed（hierarchical clustering）, but choose Inc type as classification basis
# 1.strongly 1 to 1 assocaiation
# 2.Inc type still the most common classification basis, to make the following analysis more intuitional

## Geographical(Host) distribution 

In [ ]:
#---custom---following 3 cells

lo_order<-ndm_all %>%
dplyr::count(location_all)
lo_order<-lo_order[-which(lo_order$location_all=='Missing'),]
lo_order<-na.omit(lo_order)
keep=lo_order[order(-lo_order$n),]$location_all[1:4]

lo_order<-ndm_all %>%
dplyr::count(host_all)
lo_order<-lo_order[-which(lo_order$host_all=='Missing'),]
lo_order<-na.omit(lo_order)
keep=lo_order[order(-lo_order$n),]$host_all[1:4]

lo_order<-ndm_all %>%
dplyr::count(rep2)
lo_order<-lo_order[-which(lo_order$rep2=='Missing'),]
lo_order<-na.omit(lo_order)
sel_c=lo_order[order(-lo_order$n),]$rep2[1:17]

In [ ]:
metadata<-data.frame(ACC_NUCCORE=ndm_all$ACC_NUCCORE,host=ndm_all$host_all,continent=ndm_all$Location2,country=ndm_all$location_all,Inc_type=ndm_all$rep2)

### Inc & country

In [ ]:
pl_pop_metadata<-metadata[-which((metadata$Inc_type=='Missing') | (metadata$country=='Missing')),]

#pl_pop_metadata <- subset(pl_pop_metadata, pl_pop_metadata$country %in% keep)

#pl_pop_metadata <- subset(pl_pop_metadata, pl_pop_metadata$Inc_type %in% sel_c)

pl_pop_metadata$Inc_type<-as.character(pl_pop_metadata$Inc_type)
pl_pop_metadata$country<-as.character(pl_pop_metadata$country)

# check if there is a significant association  

results_association <- NULL

for(cluster in levels(as.factor(pl_pop_metadata$Inc_type)))#####change
{
    df_cluster <- subset(pl_pop_metadata, pl_pop_metadata$Inc_type == cluster)#####change
    df_other_clusters <- subset(pl_pop_metadata, pl_pop_metadata$Inc_type != cluster)#####change
    
    df_counts_cluster <- as.data.frame(table(df_cluster$country))#####change
    df_counts_other_clusters <- as.data.frame(table(df_other_clusters$country))#####change
   
    for(category in df_counts_cluster$Var1)
    {
      particular_cluster_cat <- subset(df_counts_cluster,df_counts_cluster$Var1 == category)
      particular_cluster_other_cat <- subset(df_counts_cluster,df_counts_cluster$Var1 != category)
      sumatory_cat <- sum(as.numeric(as.character(particular_cluster_cat$Freq)))
      sumatory_other_cat <- sum(as.numeric(as.character(particular_cluster_other_cat$Freq)))
      
      vector_cluster <- c(sumatory_cat, sumatory_other_cat)
      
      dif_clusters_cat <- subset(df_counts_other_clusters,df_counts_other_clusters$Var1 == category)
      dif_clusters_dif_cat <- subset(df_counts_other_clusters,df_counts_other_clusters$Var1 != category)
      sumatory_dif_clusters_cat <- sum(as.numeric(as.character(dif_clusters_cat$Freq)))
      sumatory_dif_clusters_dif_cat <- sum(as.numeric(as.character(dif_clusters_dif_cat$Freq)))
      
      vector_other_clusters <- c(sumatory_dif_clusters_cat, sumatory_dif_clusters_dif_cat)
      
      contingency_table <<- rbind(vector_cluster, vector_other_clusters)
      colnames(contingency_table) <- c('Particular_location','Other_locations')
      rownames(contingency_table) <- c('Particular_Cluster','Other_clusters')
      
          
    result_fisher_test <- fisher.test(x=contingency_table,alternative="greater") # We perform a Fisher test using our contingency table
    
    naive_p_value <- as.numeric(as.character(result_fisher_test[1]))
    
    result <- c(cluster, category, naive_p_value)
    results_association <<- rbind(results_association,result)
    
    }
    
}

colnames(results_association) <- c('Plasmid_Inc.type','country','naive_pvalue')
results_association <- as.data.frame(results_association)
results_association$naive_pvalue <- as.numeric(as.character(results_association$naive_pvalue))

results_association$bh_correction <- p.adjust(results_association$naive_pvalue, method = 'BH', n = length(results_association$naive_pvalue)) # Correcting by Benjamini-Hochberg (BH)
results_association$bonferroni_correction <- p.adjust(results_association$naive_pvalue, method = 'bonferroni', n = length(results_association$naive_pvalue)) # Correcting by Bonferroni

results_association

subset(results_association, results_association$bh_correction < 0.05)

subset(results_association, results_association$naive_pvalue < 0.05)

clusters_association <- subset(results_association, results_association$bonferroni_correction < 0.05)

clusters_association

In [ ]:
pl_pop_metadata %>%
dplyr::count(Inc_type)

pl_pop_metadata %>%
dplyr::count(country)

### Inc & host

In [ ]:
pl_pop_metadata<-metadata[-which((metadata$Inc_type=='Missing') | (metadata$host=='Missing')),]

#pl_pop_metadata <- subset(pl_pop_metadata, pl_pop_metadata$host_all %in% keep)

#pl_pop_metadata <- subset(pl_pop_metadata, pl_pop_metadata$Inc_type %in% sel_c)

pl_pop_metadata$Inc_type<-as.character(pl_pop_metadata$Inc_type)
pl_pop_metadata$host_all<-as.character(pl_pop_metadata$host)

# check if there is a significant association  

results_association <- NULL

for(cluster in levels(as.factor(pl_pop_metadata$Inc_type)))#####change
{
    df_cluster <- subset(pl_pop_metadata, pl_pop_metadata$Inc_type == cluster)#####change
    df_other_clusters <- subset(pl_pop_metadata, pl_pop_metadata$Inc_type != cluster)#####change

    df_counts_cluster <- as.data.frame(table(df_cluster$host))#####change
    df_counts_other_clusters <- as.data.frame(table(df_other_clusters$host))#####change
    
    for(category in df_counts_cluster$Var1)
    {
      particular_cluster_cat <- subset(df_counts_cluster,df_counts_cluster$Var1 == category)
      particular_cluster_other_cat <- subset(df_counts_cluster,df_counts_cluster$Var1 != category)
      sumatory_cat <- sum(as.numeric(as.character(particular_cluster_cat$Freq)))
      sumatory_other_cat <- sum(as.numeric(as.character(particular_cluster_other_cat$Freq)))
      
      vector_cluster <- c(sumatory_cat, sumatory_other_cat)
      
      dif_clusters_cat <- subset(df_counts_other_clusters,df_counts_other_clusters$Var1 == category)
      dif_clusters_dif_cat <- subset(df_counts_other_clusters,df_counts_other_clusters$Var1 != category)
      sumatory_dif_clusters_cat <- sum(as.numeric(as.character(dif_clusters_cat$Freq)))
      sumatory_dif_clusters_dif_cat <- sum(as.numeric(as.character(dif_clusters_dif_cat$Freq)))
      
      vector_other_clusters <- c(sumatory_dif_clusters_cat, sumatory_dif_clusters_dif_cat)
      
      contingency_table <<- rbind(vector_cluster, vector_other_clusters)
      colnames(contingency_table) <- c('Particular_location','Other_locations')
      rownames(contingency_table) <- c('Particular_Cluster','Other_clusters')
      
          
    result_fisher_test <- fisher.test(x=contingency_table,alternative="greater") # We perform a Fisher test using our contingency table
    
    naive_p_value <- as.numeric(as.character(result_fisher_test[1]))
    
    result <- c(cluster, category, naive_p_value)
    results_association <<- rbind(results_association,result)
    
    }
    
}

colnames(results_association) <- c('Plasmid_Inc.type','Source','naive_pvalue')
results_association <- as.data.frame(results_association)
results_association$naive_pvalue <- as.numeric(as.character(results_association$naive_pvalue))

results_association$bh_correction <- p.adjust(results_association$naive_pvalue, method = 'BH', n = length(results_association$naive_pvalue)) # Correcting by Benjamini-Hochberg (BH)
results_association$bonferroni_correction <- p.adjust(results_association$naive_pvalue, method = 'bonferroni', n = length(results_association$naive_pvalue)) # Correcting by Bonferroni

results_association

subset(results_association, results_association$bh_correction < 0.05)

subset(results_association, results_association$naive_pvalue < 0.05)

clusters_association <- subset(results_association, results_association$bonferroni_correction < 0.05)

clusters_association

In [ ]:
pl_pop_metadata %>%
dplyr::count(Inc_type)

pl_pop_metadata %>%
dplyr::count(host)

In [ ]:
write.csv(x =ndm_all, file = './input/ndm_all_processed.csv', row.names = FALSE))

# inner pop

## TGS

In [ ]:
lo_order<-ndm_all%>%
dplyr::count(rep2)
lo_order<-lo_order[-which(lo_order$rep2=='Missing'),]
lo_order[order(-lo_order$n),]$rep2

### IncX3

In [ ]:
colnames(ndm_all)

In [ ]:
incx3<-ndm_all[which(ndm_all$rep2 == 'IncX3'),]
incx3.1<-as.data.frame(cbind(incx3$ACC_NUCCORE,incx3$rep2,incx3$Length_NUCCORE,incx3$taxon_name,incx3$host_all,incx3$Location2,incx3$subtype))
incx3.1<-unique(incx3.1)
colnames(incx3.1)=c('ACC_NUCCORE','rep2','Length_NUCCORE','taxon_name','host_all','continent','subtype')
incx3.1$Length_NUCCORE<-as.numeric(as.character(incx3.1$Length_NUCCORE))
incx3.1 %>%
dplyr::count(taxon_name)
#arrange(incx3.1,Length_NUCCORE)
#incx3.1<-incx3.1[which((incx3.1$Length_NUCCORE > 46000) & (incx3.1$Length_NUCCORE < 47000)),]
incx3.1_E<-incx3.1[which(incx3.1$taxon_name=='Escherichia coli'),]
incx3.1_E %>%
dplyr::count(subtype)
incx3.1_K<-incx3.1[which(incx3.1$taxon_name=='Klebsiella pneumoniae'),]
incx3.1_K %>%
dplyr::count(subtype)
#write.csv(x =incx3.1_E, file = './input/incx3.1_E.csv', row.names = FALSE)
#write.csv(x =incx3.1_K, file = './input/incx3.1_K.csv', row.names = FALSE)

### IncA/C

In [ ]:
incac<-ndm_all[which(ndm_all$rep2 == 'IncA/C'),]
incac.1<-as.data.frame(cbind(incac$ACC_NUCCORE,incac$rep2,incac$Length_NUCCORE,incac$taxon_name,incac$host_all,incac$Location2,incac$subtype))
incac.1<-unique(incac.1)
colnames(incac.1)=c('ACC_NUCCORE','rep2','Length_NUCCORE','taxon_name','host_all','continent','subtype')
incac.1$Length_NUCCORE<-as.numeric(as.character(incac.1$Length_NUCCORE))
incac.1  %>%
dplyr::count(taxon_name)
#arrange(incac.1,Length_NUCCORE)
#incac.1<-incac.1[which((incac.1$Length_NUCCORE > 46000) & (incac.1$Length_NUCCORE < 47000)),]
incac.1_E<-incac.1[which(incac.1$taxon_name=='Escherichia coli'),]
incac.1_E %>%
dplyr::count(subtype)
incac.1_K<-incac.1[which(incac.1$taxon_name=='Klebsiella pneumoniae'),]
incac.1_K %>%
dplyr::count(subtype)
#write.csv(x =incac.1_E, file = './input/incac.1_E.csv', row.names = FALSE)
#write.csv(x =incac.1_K, file = './input/incac.1_K.csv', row.names = FALSE)

### IncFII

In [ ]:
incf2<-ndm_all[which(ndm_all$rep2 == 'IncFII'),]
incf2.1<-as.data.frame(cbind(incf2$ACC_NUCCORE,incf2$rep2,incf2$Length_NUCCORE,incf2$taxon_name,incf2$host_all,incf2$Location2,incf2$subtype))
incf2.1<-unique(incf2.1)
colnames(incf2.1)=c('ACC_NUCCORE','rep2','Length_NUCCORE','taxon_name','host_all','continent','subtype')
incf2.1$Length_NUCCORE<-as.numeric(as.character(incf2.1$Length_NUCCORE))
incf2.1  %>%
dplyr::count(taxon_name)
#arrange(incf2.1,Length_NUCCORE)
#incf2.1<-incf2.1[which((incf2.1$Length_NUCCORE > 46000) & (incf2.1$Length_NUCCORE < 47000)),]
incf2.1_E<-incf2.1[which(incf2.1$taxon_name=='Escherichia coli'),]
incf2.1_E %>%
dplyr::count(subtype)
incf2.1_K<-incf2.1[which(incf2.1$taxon_name=='Klebsiella pneumoniae'),]
incf2.1_K %>%
dplyr::count(subtype)
#write.csv(x =incf2.1_E, file = './input/incf2.1_E.csv', row.names = FALSE)
#write.csv(x =incf2.1_K, file = './input/incf2.1_K.csv', row.names = FALSE)

### IncFIB

In [ ]:
incfib<-ndm_all[which(ndm_all$rep2 == 'IncFIB'),]
incfib.1<-as.data.frame(cbind(incfib$ACC_NUCCORE,incfib$rep2,incfib$Length_NUCCORE,incfib$taxon_name,incfib$host_all,incfib$Location2,incfib$subtype))
incfib.1<-unique(incfib.1)
colnames(incfib.1)=c('ACC_NUCCORE','rep2','Length_NUCCORE','taxon_name','host_all','continent','subtype')
incfib.1$Length_NUCCORE<-as.numeric(as.character(incfib.1$Length_NUCCORE))
incfib.1  %>%
dplyr::count(taxon_name)
#arrange(incfib.1,Length_NUCCORE)
#incfib.1<-incfib.1[which((incfib.1$Length_NUCCORE > 46000) & (incfib.1$Length_NUCCORE < 47000)),]
incfib.1_E<-incfib.1[which(incfib.1$taxon_name=='Escherichia coli'),]
incfib.1_E %>%
dplyr::count(subtype)
incfib.1_K<-incfib.1[which(incfib.1$taxon_name=='Klebsiella pneumoniae'),]
incfib.1_K %>%
dplyr::count(subtype)

### IncFIA

In [ ]:
incfia<-ndm_all[which(ndm_all$rep2 == 'IncFIA'),]
incfia.1<-as.data.frame(cbind(incfia$ACC_NUCCORE,incfia$rep2,incfia$Length_NUCCORE,incfia$taxon_name,incfia$host_all,incfia$Location2,incfia$subtype))
incfia.1<-unique(incfia.1)
colnames(incfia.1)=c('ACC_NUCCORE','rep2','Length_NUCCORE','taxon_name','host_all','continent','subtype')
incfia.1$Length_NUCCORE<-as.numeric(as.character(incfia.1$Length_NUCCORE))
incfia.1 %>%
dplyr::count(host_all)
#arrange(incfia.1,Length_NUCCORE)
#incfia.1<-incfia.1[which((incfia.1$Length_NUCCORE > 46000) & (incfia.1$Length_NUCCORE < 47000)),]
incfia.1_E<-incfia.1[which(incfia.1$taxon_name=='Escherichia coli'),]
incfia.1_E %>%
dplyr::count(subtype)

## add NGS data

### E.coli

In [ ]:
prediction_contigs_raw <- read.csv('./input/prediction_contigs_raw_merge.csv', header = T,stringsAsFactors=F)#predicted by mlplasmid
Assembly_metadata<-read.table('./input/microbigge.tsv',sep="\t",header=T,row.names=NULL,quote="",stringsAsFactors=F)
Assembly_metadata$Contig<-gsub(pattern='NZ_',replacement='',Assembly_metadata$Contig)

In [ ]:
colnames(prediction_contigs_raw)
colnames(Assembly_metadata)

In [ ]:
colnames(Assembly_metadata)[5]<-'contig'

In [ ]:
prediction_contigs <- left_join(Assembly_metadata,unique(prediction_contigs_raw),by='contig',copy=TRUE)
dim(prediction_contigs)
colnames(prediction_contigs)

In [ ]:
cols95 <- c('Isolation.type' , 'Isolation.source','Host')
prediction_contigs$isolation_all <- apply( prediction_contigs[ , cols95 ] , 1 , paste , collapse = "," )

prediction_contigs$Host[is.na(prediction_contigs$Host)]<-prediction_contigs$Isolation.source[is.na(prediction_contigs$Host)]
prediction_contigs$Host[which(prediction_contigs$Host=='')]<-prediction_contigs$Isolation.source[which(prediction_contigs$Host=='')]
prediction_contigs$Host[which(prediction_contigs$Host=='Not Available')]<-prediction_contigs$Isolation.source[which(prediction_contigs$Host=='Not Available')]
prediction_contigs$Host[which(prediction_contigs$Host=='not collected')]<-prediction_contigs$Isolation.source[which(prediction_contigs$Host=='not collected')]

prediction_contigs$Host[is.na(prediction_contigs$Host)]<-prediction_contigs$Isolation.type[is.na(prediction_contigs$Host)]
prediction_contigs$Host[which(prediction_contigs$Host=='')]<-prediction_contigs$Isolation.type[which(prediction_contigs$Host=='')]

prediction_contigs$Host[is.na(prediction_contigs$Host)]<-'Missing'

prediction_contigs$Host1<-prediction_contigs$Host

In [ ]:
prediction_contigs$Host[which(grepl('clinical|patient|human|lung|Homo|blood|urine', prediction_contigs$Host,ignore.case = TRUE))] <- 'Homo sapiens'
#prediction_contigs$Host[which(grepl('fly|cat|chicken|dog|duck|cattle|pig|swine|poultry|swab|feces', prediction_contigs$Host,ignore.case = TRUE))] <- 'Animal'
prediction_contigs$Host[which(grepl('leaf|spinach', prediction_contigs$Host,ignore.case = TRUE))] <- 'Vegetable'
prediction_contigs$Host[which(grepl('Chicken manure compost|feed|Siphon', prediction_contigs$Host,ignore.case = TRUE))] <- 'Breeding environment'
prediction_contigs$Host[which(grepl('sewage from slaughterhouse', prediction_contigs$Host,ignore.case = TRUE))] <- 'Slaughtering environment'
prediction_contigs$Host[which(grepl('Dog Environment', prediction_contigs$Host,ignore.case = TRUE))] <- 'Pet environment'
prediction_contigs$Host[which(grepl('washroom sink in hospital intensive care|medical sewage', prediction_contigs$Host,ignore.case = TRUE))] <- 'Hospital environment'
prediction_contigs$Host[which(grepl('supermarket|meat|milk|market', prediction_contigs$Host,ignore.case = TRUE))] <- 'Animal food'
prediction_contigs$Host[which(grepl('chicken|duck|poultry|Broiler', prediction_contigs$Host,ignore.case = TRUE))] <- 'Poultry'
prediction_contigs$Host[which(grepl('pig|swine', prediction_contigs$Host,ignore.case = TRUE))] <- 'Swine'
prediction_contigs$Host[which(grepl('Equus caballus', prediction_contigs$Host,ignore.case = TRUE))] <- 'Horse'
prediction_contigs$Host[which(grepl('fly', prediction_contigs$Host,ignore.case = TRUE))] <- 'Fly'
prediction_contigs$Host[which(grepl('cattle', prediction_contigs$Host,ignore.case = TRUE))] <- 'Cattle'
#prediction_contigs$Host[which(grepl('swallow', prediction_contigs$Host,ignore.case = TRUE))] <- 'Swallow'
prediction_contigs$Host[which(grepl('Spheniscus magellanicus|swallow', prediction_contigs$Host,ignore.case = TRUE))] <- 'Wild animal'
prediction_contigs$Host[which(grepl('Canis lupus familiaris|Companion animal|dog', prediction_contigs$Host,ignore.case = TRUE))] <- 'Pet'
prediction_contigs$Host[which(grepl('Sewage|water|environment|^environmental$|soil|Door|counter|environmental sample|floor|rug|shoes|computer|wood siding|Thermometer|yard|pump|frame', prediction_contigs$Host,ignore.case = TRUE))] <- 'Environment'
prediction_contigs$Host[which(grepl('swab|feces|not|other|Chlorohexidine|ceftazidime', prediction_contigs$Host,ignore.case = TRUE))] <- 'Missing'

#prediction_contigs=prediction_contigs %>% drop_na(Element.symbol)

prediction_contigs %>%
    dplyr::count(Host)

In [ ]:
prediction_contigs$Location<-str_split_fixed(prediction_contigs$Location,":",2)[,1]

prediction_contigs$Location[which(grepl('China|Hong|Taiwan', prediction_contigs$Location,ignore.case = TRUE))] <- 'China'
prediction_contigs$Location<-gsub('"','',prediction_contigs$Location)
prediction_contigs$Location[is.na(prediction_contigs$Location)]<-'Missing'
prediction_contigs$Location[which(prediction_contigs$Location=='')]<-'Missing'
prediction_contigs$Location[which(prediction_contigs$Location=='not collected')]<-'Missing'

prediction_contigs$Location2<-prediction_contigs$Location

In [ ]:
prediction_contigs$Location2[which(grepl('Afghanistan|Armenia|Azerbaijan|Bahrain|Viet Nam|Bangladesh|Bhutan|Brunei|Cambodia|China|Cyprus|East Timor|Georgia|India|Indonesia|Iran|Iraq|Israel|Japan|Jordan|Kazakhstan|Kuwait|Kyrgyzstan|Laos|Lebanon|Malaysia|Maldives|Mongolia|Myanmar|Nepal|North Korea|Oman|Pakistan|State of Palestine|Jerusalem|Philippines|Qatar|Russia|Saudi Arabia|Singapore|South Korea|Sri Lanka|Syria|Taiwan|Tajikistan|Thailand|Turkey|Turkmenistan|United Arab|Uzbekistan|Vietnam|Yemen', prediction_contigs$Location,ignore.case = TRUE))] <- 'Asia'
prediction_contigs$Location2[which(grepl('Albania|Andorra|Armenia|Austria|Azerbaijan|Belarus|Belgium|Bosnia and Herzegovina|Bulgaria|Croatia|Cyprus|Czech Republic|Denmark|Estonia|Finland|France|Georgia|Germany|Greece|Hungary|Iceland|Republic of Ireland|Italy|Kazakhstan|Latvia|Liechtenstein|Lithuania|Luxembourg|Malta|Moldova|Monaco|Montenegro|Netherlands|North Macedonia|Norway|Poland|Portugal|Romania|Russia|San Marino|Serbia|Slovakia|Slovenia|Spain|Sweden|Switzerland|Turkey|Ukraine|United Kingdom|Vatican City', prediction_contigs$Location,ignore.case = TRUE))] <- 'Europe'
prediction_contigs$Location2[which(grepl('Nigeria|South Africa|Egypt|Algeria|Angola|Morocco|Kenya|Ethiopia|Libya|Ghana|Malawi|Niger', prediction_contigs$Location,ignore.case = TRUE))] <- 'Africa'
prediction_contigs$Location2[which(grepl('Anguilla|Antigua and Barbuda|Aruba|Bahamas|Barbados|Belize|Bermuda|Bonaire|British Virgin Islands|Canada|Cayman Islands|France|Costa Rica|Cuba|Curaçao|Dominica|Dominican Republic|El Salvador|Venezuela|Greenland|Grenada|France|Guantanamo Bay Naval Base|Guatemala|Haiti|Honduras|Jamaica|France|Mexico|Montserrat|Brades|Nicaragua|Nueva Esparta|Panama|Puerto Rico|Saba|San Andrés and Providencia|Saint Barthélemy|Saint Kitts and Nevis|Saint Lucia|Saint Martin|Saint Pierre and Miquelon|Saint Vincent and the Grenadines|Sint Eustatius|Sint Maarten|Trinidad and Tobago|Turks and Caicos Islands|United States|United States Virgin Islands|USA', prediction_contigs$Location,ignore.case = TRUE))] <- 'North America'
prediction_contigs$Location2[which(grepl('Argentina|Bolivia|Sucre|Norway|Brazil|Chile|Colombia|Ecuador|Falkland Islands|French Guiana|Guyana|Paraguay|Peru|South Georgia and the South Sandwich Islands|Suriname|Uruguay|Venezuela', prediction_contigs$Location,ignore.case = TRUE))] <- 'South America'
prediction_contigs$Location2[which(grepl('Australia|Fiji|Kiribati|Marshall Islands|Majuro|Micronesia|Nauru|New Zealand|Palau|Papua New Guinea|Samoa|Solomon Islands|Tonga|Tuvalu|Vanuatu', prediction_contigs$Location,ignore.case = TRUE))] <- 'Australia'

prediction_contigs %>%
    dplyr::count(Location2)

In [ ]:
cols96 <- c('rep_type.s.','product')
prediction_contigs$rep <- apply( prediction_contigs[ , cols96 ] , 1 , paste , collapse = "," )

In [ ]:
prediction_contigs$rep <- str_split_fixed(prediction_contigs$rep, "\\,", 2)[,1]

In [ ]:
prediction_contigs %>%
    dplyr::count(rep)

In [ ]:
predicted_plasmid_contigs<-prediction_contigs[-which(prediction_contigs$rep %in% c('NA','-')),] 
dim(predicted_plasmid_contigs)
predicted_chr_contigs <- subset(prediction_contigs,prediction_contigs$Prediction=='Chromosome')
dim(predicted_chr_contigs)

In [ ]:
predicted_plasmid_contigs_del<-predicted_plasmid_contigs[-which(duplicated(predicted_plasmid_contigs$Assembly.x)),]
predicted_chr_contigs_del<-predicted_chr_contigs[-which(duplicated(predicted_chr_contigs$Assembly.x)),]
dim(predicted_plasmid_contigs_del)
dim(predicted_chr_contigs_del)

In [ ]:
del2p<-predicted_plasmid_contigs[duplicated(predicted_plasmid_contigs$Assembly.x),]$Assembly.x
del2c<-predicted_chr_contigs[duplicated(predicted_chr_contigs$Assembly.x),]$Assembly.x
predicted_plasmid_contigs_del2<-predicted_plasmid_contigs[-which(predicted_plasmid_contigs$Assembly.x %in% del2p),]
predicted_chr_contigs_del2<-predicted_chr_contigs[-which(predicted_chr_contigs$Assembly.x %in% del2c),]
dim(predicted_plasmid_contigs_del2)
dim(predicted_chr_contigs_del2)

In [ ]:
predicted_plasmid_contigs_del %>%
    dplyr::count(rep)

In [ ]:
predicted_plasmid_contigs_del %>% 
  dplyr::group_by(Element.symbol.x) %>% 
  dplyr::summarise(Count=n())

In [ ]:
predicted_chr_contigs_del %>% 
  dplyr::group_by(Element.symbol.x) %>% 
  dplyr::summarise(Count=n())

#### IncX3

In [ ]:
a_incx3<-predicted_plasmid_contigs_del[which(predicted_plasmid_contigs_del$rep == 'IncX3'),]
a_incx3.1<-as.data.frame(cbind(a_incx3$Assembly.x,a_incx3$rep,a_incx3$Contig_length,a_incx3$Host,a_incx3$contig,a_incx3$Location2,a_incx3$Element.symbol.x))
a_incx3.1<-unique(a_incx3.1)
colnames(a_incx3.1)=c('Assembly_number','Inc_type','Length','Host','Contig','Continent','Subtype')
a_incx3.1$Length<-as.numeric(as.character(a_incx3.1$Length))
#arrange(a_incx3.1,Length)
a_incx3.2<-a_incx3.1[which((a_incx3.1$Subtype == 'blaNDM-5')),]
#arrange(a_incx3.2,Length)
dim(a_incx3.1)
#write.csv(x =a_incx3.1, file = './input/a_incx3.1_E.csv', row.names = FALSE)
#write.csv(x =a_incx3.2, file = './input/a_incx3.2_E.csv', row.names = FALSE)

a_incx3.1 %>%
dplyr::count(Continent)

a_incx3.1 %>%
dplyr::count(Subtype)

a_incx3.1 %>%
dplyr::count(Host)

#### IncFIA

In [ ]:
a_incfia<-predicted_plasmid_contigs_del[which(predicted_plasmid_contigs_del$rep == 'IncFIA'),]
a_incfia.1<-as.data.frame(cbind(a_incfia$Assembly.x,a_incfia$rep,a_incfia$Contig_length,a_incfia$Host,a_incfia$contig,a_incfia$Location2,a_incfia$Element.symbol.x))
a_incfia.1<-unique(a_incfia.1)
colnames(a_incfia.1)=c('Assembly_number','Inc_type','Length','Host','Contig','Continent','Subtype')
a_incfia.1$Length<-as.numeric(as.character(a_incfia.1$Length))

a_incfia.1 %>%
dplyr::count(Continent)

a_incfia.1 %>%
dplyr::count(Host)

a_incfia.1 %>%
dplyr::count(Subtype)
#arrange(a_incfia.1,Length)
a_incfia.2<-a_incfia.1[which((a_incfia.1$Subtype == 'blaNDM-5')),]

#write.csv(x =a_incfia.1, file = './input/a_incfia.1_E.csv', row.names = FALSE)

#### IncA/C

In [ ]:
a_incc<-predicted_plasmid_contigs_del[which(predicted_plasmid_contigs_del$rep == 'IncC'),]
a_incc.1<-as.data.frame(cbind(a_incc$Assembly.x,a_incc$rep,a_incc$Contig_length,a_incc$Host,a_incc$contig,a_incc$Location2,a_incc$Element.symbol.x))
a_incc.1<-unique(a_incc.1)
colnames(a_incc.1)=c('Assembly_number','Inc_type','Length','Host','Contig','Continent','Subtype')
a_incc.1$Length<-as.numeric(as.character(a_incc.1$Length))
a_incc.1 %>%
dplyr::count(Subtype)

a_incc.1 %>%
dplyr::count(Continent)

a_incc.1 %>%
dplyr::count(Host)
#arrange(a_incc.1,Length)
#write.csv(x =a_incc.1, file = './input/a_incc.1_E.csv', row.names = FALSE)

### Klebsiella pneumoniae

In [ ]:
ndm_contig_k_type <- read.csv('./input/ndm_contig_k_type.csv', header = T,stringsAsFactors=F)#predicted by mlplasmid
assemble_all_contigs_k <- read.csv('./input/assemble_all_contigs_k.csv', header = T,stringsAsFactors=F)
k_Assembly_metadata<-read.table('./input/microbigge_k.tsv',sep="\t",header=T,row.names=NULL,quote="",stringsAsFactors=F)

In [ ]:
k_Assembly_metadata$Contig<-gsub(pattern='NZ_',replacement='',k_Assembly_metadata$Contig)

In [ ]:
assemble_all_contigs_k$Contig_name<-str_split_fixed(assemble_all_contigs_k$Contig_name," ",2)[,1]

In [ ]:
colnames(ndm_contig_k_type)
colnames(assemble_all_contigs_k)
colnames(k_Assembly_metadata)

In [ ]:
head(ndm_contig_k_type)
head(assemble_all_contigs_k)
head(k_Assembly_metadata)

In [ ]:
colnames(ndm_contig_k_type)[1]<-'Contig_name'
colnames(k_Assembly_metadata)[9]<-'Contig_name'

In [ ]:
k_prediction_contigs_raw<-left_join(assemble_all_contigs_k,unique(ndm_contig_k_type),by='Contig_name')

In [ ]:
k_prediction_contigs1 <- left_join(k_Assembly_metadata,unique(k_prediction_contigs_raw),by='Contig_name')

In [ ]:
mapping_type <- read.csv('./input/k_mapping_type.csv', header = T,stringsAsFactors=F)

c_type<-subset(ndm_all,select=c('ACC_NUCCORE','rep2','mob','mpf'))
k_mapping_type<-left_join(mapping_type,unique(c_type),by='ACC_NUCCORE')

k_prediction_contigs<-left_join(k_prediction_contigs1,unique(k_mapping_type),by='Contig_name')
dim(k_prediction_contigs)
colnames(k_prediction_contigs)

In [ ]:
cols95 <- c('Isolation.type' , 'Isolation.source','Host')
k_prediction_contigs$isolation_all <- apply( k_prediction_contigs[ , cols95 ] , 1 , paste , collapse = "," )

k_prediction_contigs$Host[is.na(k_prediction_contigs$Host)]<-k_prediction_contigs$Isolation.source[is.na(k_prediction_contigs$Host)]
k_prediction_contigs$Host[which(k_prediction_contigs$Host=='')]<-k_prediction_contigs$Isolation.source[which(k_prediction_contigs$Host=='')]
k_prediction_contigs$Host[which(k_prediction_contigs$Host=='Not Available')]<-k_prediction_contigs$Isolation.source[which(k_prediction_contigs$Host=='Not Available')]
k_prediction_contigs$Host[which(k_prediction_contigs$Host=='not collected')]<-k_prediction_contigs$Isolation.source[which(k_prediction_contigs$Host=='not collected')]

k_prediction_contigs$Host[is.na(k_prediction_contigs$Host)]<-k_prediction_contigs$Isolation.type[is.na(k_prediction_contigs$Host)]
k_prediction_contigs$Host[which(k_prediction_contigs$Host=='')]<-k_prediction_contigs$Isolation.type[which(k_prediction_contigs$Host=='')]

k_prediction_contigs$Host[is.na(k_prediction_contigs$Host)]<-'Missing'

k_prediction_contigs$Host1<-k_prediction_contigs$Host

In [ ]:
k_prediction_contigs$Host[which(grepl('clinical|patient|human|lung|Homo|blood|urine|hospital', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Homo sapiens'
#k_prediction_contigs$Host[which(grepl('fly|cat|chicken|dog|duck|cattle|pig|swine|poultry|swab|feces', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Animal'
k_prediction_contigs$Host[which(grepl('leaf|spinach', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Vegetable'
k_prediction_contigs$Host[which(grepl('Chicken manure compost|feed|Siphon', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Breeding environment'
k_prediction_contigs$Host[which(grepl('sewage from slaughterhouse', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Slaughtering environment'
k_prediction_contigs$Host[which(grepl('Dog Environment', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Pet environment'
k_prediction_contigs$Host[which(grepl('washroom sink in hospital intensive care|medical sewage', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Hospital environment'
k_prediction_contigs$Host[which(grepl('supermarket|meat|milk|market', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Animal food'
k_prediction_contigs$Host[which(grepl('chicken|duck|poultry|Broiler', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Poultry'
k_prediction_contigs$Host[which(grepl('pig|swine', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Swine'
k_prediction_contigs$Host[which(grepl('Equus caballus', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Horse'
k_prediction_contigs$Host[which(grepl('fly', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Fly'
k_prediction_contigs$Host[which(grepl('cattle', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Cattle'
#k_prediction_contigs$Host[which(grepl('swallow', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Swallow'
k_prediction_contigs$Host[which(grepl('Spheniscus magellanicus|swallow|bird', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Wild animal'
k_prediction_contigs$Host[which(grepl('Canis lupus familiaris|Companion animal|dog', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Pet'
k_prediction_contigs$Host[which(grepl('Sewage|water|environment|^environmental$|soil|Door|counter|enviromental|natural|river|sink|fodder|floor|rug|shoes|computer|wood siding|Thermometer|yard|pump|frame', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Environment'
k_prediction_contigs$Host[which(grepl('swab|feces|not|other|Chlorohexidine|ceftazidime', k_prediction_contigs$Host,ignore.case = TRUE))] <- 'Missing'

#k_prediction_contigs=k_prediction_contigs %>% drop_na(Element.symbol)

k_prediction_contigs %>%
    dplyr::count(Host)

In [ ]:
k_prediction_contigs$Location<-str_split_fixed(k_prediction_contigs$Location,":",2)[,1]

k_prediction_contigs$Location[which(grepl('China|Hong|Taiwan', k_prediction_contigs$Location,ignore.case = TRUE))] <- 'China'
k_prediction_contigs$Location<-gsub('"','',k_prediction_contigs$Location)
k_prediction_contigs$Location[is.na(k_prediction_contigs$Location)]<-'Missing'
k_prediction_contigs$Location[which(k_prediction_contigs$Location=='')]<-'Missing'
k_prediction_contigs$Location[which(k_prediction_contigs$Location=='not collected')]<-'Missing'

In [ ]:
k_prediction_contigs$Location2<-k_prediction_contigs$Location

In [ ]:
k_prediction_contigs$Location2[which(grepl('Korea|Afghanistan|Armenia|Azerbaijan|Bahrain|Viet Nam|Bangladesh|Bhutan|Brunei|Cambodia|China|Cyprus|East Timor|Georgia|India|Indonesia|Iran|Iraq|Israel|Japan|Jordan|Kazakhstan|Kuwait|Kyrgyzstan|Laos|Lebanon|Malaysia|Maldives|Mongolia|Myanmar|Nepal|North Korea|Oman|Pakistan|State of Palestine|Jerusalem|Philippines|Qatar|Russia|Saudi Arabia|Singapore|South Korea|Sri Lanka|Syria|Taiwan|Tajikistan|Thailand|Turkey|Turkmenistan|United Arab|Uzbekistan|Vietnam|Yemen', k_prediction_contigs$Location,ignore.case = TRUE))] <- 'Asia'
k_prediction_contigs$Location2[which(grepl('Ireland|Albania|Andorra|Armenia|Austria|Azerbaijan|Belarus|Belgium|Bosnia and Herzegovina|Bulgaria|Croatia|Cyprus|Czech Republic|Denmark|Estonia|Finland|France|Georgia|Germany|Greece|Hungary|Iceland|Republic of Ireland|Italy|Kazakhstan|Latvia|Liechtenstein|Lithuania|Luxembourg|Malta|Moldova|Monaco|Montenegro|Netherlands|North Macedonia|Norway|Poland|Portugal|Romania|Russia|San Marino|Serbia|Slovakia|Slovenia|Spain|Sweden|Switzerland|Turkey|Ukraine|United Kingdom|Vatican City', k_prediction_contigs$Location,ignore.case = TRUE))] <- 'Europe'
k_prediction_contigs$Location2[which(grepl('Nigeria|South Africa|Egypt|Algeria|Angola|Morocco|Kenya|Ethiopia|Libya|Ghana|Malawi|Niger|Tunisia', k_prediction_contigs$Location,ignore.case = TRUE))] <- 'Africa'
k_prediction_contigs$Location2[which(grepl('Anguilla|Antigua and Barbuda|Aruba|Bahamas|Barbados|Belize|Bermuda|Bonaire|British Virgin Islands|Canada|Cayman Islands|France|Costa Rica|Cuba|Curaçao|Dominica|Dominican Republic|El Salvador|Venezuela|Greenland|Grenada|France|Guantanamo Bay Naval Base|Guatemala|Haiti|Honduras|Jamaica|France|Mexico|Montserrat|Brades|Nicaragua|Nueva Esparta|Panama|Puerto Rico|Saba|San Andrés and Providencia|Saint Barthélemy|Saint Kitts and Nevis|Saint Lucia|Saint Martin|Saint Pierre and Miquelon|Saint Vincent and the Grenadines|Sint Eustatius|Sint Maarten|Trinidad and Tobago|Turks and Caicos Islands|United States|United States Virgin Islands|USA', k_prediction_contigs$Location,ignore.case = TRUE))] <- 'North America'
k_prediction_contigs$Location2[which(grepl('Argentina|Bolivia|Sucre|Norway|Brazil|Chile|Colombia|Ecuador|Falkland Islands|French Guiana|Guyana|Paraguay|Peru|South Georgia and the South Sandwich Islands|Suriname|Uruguay|Venezuela', k_prediction_contigs$Location,ignore.case = TRUE))] <- 'South America'
k_prediction_contigs$Location2[which(grepl('Australia|Fiji|Kiribati|Marshall Islands|Majuro|Micronesia|Nauru|New Zealand|Palau|Papua New Guinea|Samoa|Solomon Islands|Tonga|Tuvalu|Vanuatu', k_prediction_contigs$Location,ignore.case = TRUE))] <- 'Australia'

k_prediction_contigs %>%
    dplyr::count(Location2)

In [ ]:
k_prediction_contigs %>%
    dplyr::count(rep2)

In [ ]:
k_predicted_plasmid_contigs<-k_prediction_contigs[which( !(k_prediction_contigs$rep2 %in% c('NA','Missing')) & (k_prediction_contigs$Prediction == 'Plasmid') &
                                                       !(is.na(k_prediction_contigs$rep2)) ),] 

In [ ]:
dim(k_predicted_plasmid_contigs)

In [ ]:
k_predicted_chr_contigs <- subset(k_prediction_contigs,k_prediction_contigs$Prediction=='Chromosome')

In [ ]:
dim(k_predicted_chr_contigs)

In [ ]:
k_predicted_plasmid_contigs %>%
    dplyr::count(rep2)

In [ ]:
k_predicted_plasmid_contigs%>% 
  dplyr::group_by(Element.symbol) %>% 
  dplyr::summarise(Count=n())

k_predicted_plasmid_contigs %>% 
  dplyr::group_by(Host) %>% 
  dplyr::summarise(Count=n())

k_predicted_plasmid_contigs %>% 
  dplyr::group_by(Location2) %>% 
  dplyr::summarise(Count=n())

#### IncX3

In [ ]:
k_a_incx3<-k_predicted_plasmid_contigs[which(k_predicted_plasmid_contigs$rep2 == 'IncX3'),]

k_a_incx3.1<-as.data.frame(cbind(k_a_incx3$Assembly,k_a_incx3$rep2,k_a_incx3$Contig_length,k_a_incx3$Host,k_a_incx3$Contig_name,k_a_incx3$Location2,k_a_incx3$Element.symbol))
k_a_incx3.1<-unique(k_a_incx3.1)
colnames(k_a_incx3.1)=c('Assembly_number','Inc_type','Length','Host','Contig','Continent','Subtype')
k_a_incx3.1$Length<-as.numeric(as.character(k_a_incx3.1$Length))
#arrange(k_a_incx3.1,Length)
k_a_incx3.2<-k_a_incx3.1[which((k_a_incx3.1$Subtype == 'blaNDM-5')),]
#arrange(k_a_incx3.2,Length)
dim(k_a_incx3.1)
#write.csv(x =k_a_incx3.1, file = './input/a_incx3.1_K.csv', row.names = FALSE)
#write.csv(x =k_a_incx3.2, file = './input/a_incx3.2_K.csv', row.names = FALSE)

k_a_incx3.1 %>%
dplyr::count(Continent)

k_a_incx3.1 %>%
dplyr::count(Subtype)

k_a_incx3.1 %>%
dplyr::count(Host)

#### IncFII

In [ ]:
k_a_incfii<-k_predicted_plasmid_contigs[which(k_predicted_plasmid_contigs$rep2 == 'IncFII'),]

k_a_incfii.1<-as.data.frame(cbind(k_a_incfii$Assembly,k_a_incfii$rep2,k_a_incfii$Contig_length,k_a_incfii$Host,k_a_incfii$Contig_name,k_a_incfii$Location2,k_a_incfii$Element.symbol))
k_a_incfii.1<-unique(k_a_incfii.1)
colnames(k_a_incfii.1)=c('Assembly_number','Inc_type','Length','Host','Contig','Continent','Subtype')
k_a_incfii.1$Length<-as.numeric(as.character(k_a_incfii.1$Length))
#arrange(k_a_incfii.1,Length)
#k_a_incfii.2<-k_a_incfii.1[which((k_a_incfii.1$Subtype == 'blaNDM-5')),]
#arrange(k_a_incfii.2,Length)
dim(k_a_incfii.1)
#write.csv(x =k_a_incfii.1, file = './input/a_incfii.1_K.csv', row.names = FALSE)
#write.csv(x =k_a_incfii.2, file = './input/a_incfii.2_K.csv', row.names = FALSE)

k_a_incfii.1 %>%
dplyr::count(Continent)

k_a_incfii.1 %>%
dplyr::count(Subtype)

k_a_incfii.1 %>%
dplyr::count(Host)

#### IncA/C

In [ ]:
k_a_incac<-k_predicted_plasmid_contigs[which(k_predicted_plasmid_contigs$rep2 == 'IncA/C'),]

k_a_incac.1<-as.data.frame(cbind(k_a_incac$Assembly,k_a_incac$rep2,k_a_incac$Contig_length,k_a_incac$Host,k_a_incac$Contig_name,k_a_incac$Location2,k_a_incac$Element.symbol))
k_a_incac.1<-unique(k_a_incac.1)
colnames(k_a_incac.1)=c('Assembly_number','Inc_type','Length','Host','Contig','Continent','Subtype')
k_a_incac.1$Length<-as.numeric(as.character(k_a_incac.1$Length))
#arrange(k_a_incac.1,Length)
#k_a_incac.2<-k_a_incac.1[which((k_a_incac.1$Subtype == 'blaNDM-5')),]
#arrange(k_a_incac.2,Length)
dim(k_a_incac.1)
#write.csv(x =k_a_incac.1, file = './input/a_incac.1_K.csv', row.names = FALSE)
#write.csv(x =k_a_incac.2, file = './input/a_incac.2_K.csv', row.names = FALSE)

k_a_incac.1 %>%
dplyr::count(Continent)

k_a_incac.1 %>%
dplyr::count(Subtype)

k_a_incac.1 %>%
dplyr::count(Host)

## DAPC

### IncX3-Ecoli(TGS+NGS——Conserved region)

In [ ]:
vcf_file='./input/e_incx3.1/parsnp.vcf'
pop_file='./input/a_incx3.1_E.csv'

ndm_vcf <- read.vcfR(vcf_file)
ndm_pl_gd <- vcfR2genind(ndm_vcf)
ndm_pl_gd

In [ ]:
ndm_pl_gd_ind<-as.data.frame(indNames(ndm_pl_gd))

In [ ]:
ndm_pl_gd_ind[,1]<-as.character(ndm_pl_gd_ind[,1])

#ndm_pl_gd_new1_ind.names[,1]<-gsub('.*plasmid_','',ndm_pl_gd_new1_ind.names[,1])
ndm_pl_gd_ind[,1]<-gsub('.fast.*','',ndm_pl_gd_ind[,1])
ndm_pl_gd_ind[,1]<-gsub('genomic_plasmid_','',ndm_pl_gd_ind[,1])
ndm_pl_gd_ind[,1]<-gsub('GCA_.*_.*_','',ndm_pl_gd_ind[,1])
#ndm_pl_gd_ind[,1]<-gsub('IncX3_','',ndm_pl_gd_ind[,1])

In [ ]:
colnames(ndm_pl_gd_ind)<-'Contig'

In [ ]:
ndm_pl_popRaw <- read.csv(pop_file, header = TRUE,stringsAsFactors = FALSE)

ndm_pl_pop<-left_join(ndm_pl_gd_ind,unique(ndm_pl_popRaw),by='Contig')

#ndm_pl_pop[which((ndm_pl_pop$Host=='Pet') | (ndm_pl_pop$Host=='Poultry') | (ndm_pl_pop$Host=='Swine')),]$Host<-'Animals'
#ndm_pl_pop[which(!(ndm_pl_pop$Host=='Homo sapiens')),]$Host<-'Others'

indNames(ndm_pl_gd)<-as.vector(ndm_pl_gd_ind[,1])

pop(ndm_pl_gd)

In [ ]:
pop(ndm_pl_gd)<-ndm_pl_pop$Host

pop(ndm_pl_gd)

In [ ]:
as.data.frame(pop(ndm_pl_gd)) %>%    
  dplyr::group_by(pop(ndm_pl_gd))%>%
  dplyr::summarize(Count = n())

In [ ]:
select<-as.character(ndm_pl_pop[which(ndm_pl_pop$Host %in% c('Homo sapiens','Swine','Poultry','Environment','Fly','Pet'
                                                            )),][['Contig']])

In [ ]:
select

In [ ]:
ndm_pl_gd_new1 <- ndm_pl_gd[row.names(ndm_pl_gd@tab) %in% select]

In [ ]:
ndm_pl_gd_new1

In [ ]:
#pdf("./output/fig/dapc.ndm367gl_new2.pdf")
dapc.ndm_pl_gd_new1<- dapc(ndm_pl_gd_new1, var.contrib = TRUE, scale = FALSE, n.pca = 20, n.da = nPop(ndm_pl_gd_new1) - 1)
#The scatter() function is part of the ade4 package and plots results of a DAPC analysis.
scatter(dapc.ndm_pl_gd_new1, cell = 0, pch = 18:23, cstar = 0, mstree = TRUE, lwd = 2, lty = 2,posi.da="bottomright")
#dev.off()

In [ ]:
dapc.ndm_pl_gd_new1.1 <- dapc(ndm_pl_gd_new1, n.pca = 20, n.da = 4)

cols <- brewer.pal(n =7, name = "Dark2")
cols

#pdf('./output/fig/ndm_pl_gd_new1.1.pdf')
scatter(dapc.ndm_pl_gd_new1.1, col = c('#7570B3','#D95F02','#E7298A','#66A61E','#E6AB02'), posi.da = "topright",cex = 2, legend = TRUE, clabel = F, posi.leg = "bottomleft", cleg = 1.2)
#scree.pca = TRUE, posi.pca = "topleft"
#dev.off()

In [ ]:
myCol <- c("darkblue","purple","green","orange","red","blue")
scatter(dapc.ndm_pl_gd_new1.1, posi.da="bottomright", bg="white",
pch=17:22, cstar=0, col=myCol, scree.pca=TRUE,
posi.pca="bottomleft")

In [ ]:
scatter(dapc.ndm_pl_gd_new1.1, scree.da=FALSE, bg="white", pch=20, cell=0, cstar=0, col=myCol, solid=.4,
cex=3,clab=0, leg=TRUE, txt.leg=paste("Cluster",1:4))

In [ ]:
dapc.ndm_pl_gd_new1.1$var.contr

In [ ]:
set.seed(4)
contrib <- loadingplot(dapc.ndm_pl_gd_new1.1$var.contr, axis = 2, thres = 0.07, lab.jitter = 1)

In [ ]:
freqE <- tab(genind2genpop(ndm_pl_gd_new1[loc=c("GCTTGGCCTG_TCCGAAGATG")]),freq=TRUE)

In [ ]:
matplot(freqE, pch=c("c","t"), type="b", xlab="host",
ylab="allele frequency", xaxt="n", cex=1.5,
main="SNP # E")
axis(side=1, at=1:4, lab=c("Homo Sapien","Pet","Poultry","Swine"))

In [ ]:
#dapc.ndm55_del<- dapc(ndm55gd_del, var.contrib = TRUE, scale = FALSE, n.pca = 30, n.da = nPop(ndm55gd_del) - 1)

#scatter(dapc.ndm55_del,1,1,bg="white",scree.da=FAL...%0ArkJggg==SE, legend=TRUE,solid=.4)

#set.seed(4)
#contrib_del <- loadingplot(dapc.ndm55_del$var.contr, axis = 1, thres = 0.07, lab.jitter = 1)

#write.csv(x=dapc.ndm55$var.contr,file='./output/ndm55class2snp.csv',row.names=T)

freq1 <- tab(genind2genpop(ndm_pl_gd_new1[loc=c("ATACGATCAC_CCGCTATGGA")]),freq=TRUE)

freq2 <- tab(genind2genpop(ndm_pl_gd_new1[loc=c("TATAAATGCG_GTTCATTTTC")]),freq=TRUE)

freq3 <- tab(genind2genpop(ndm_pl_gd_new1[loc=c("CCGGCGTTCC_GAACTGCTGG")]),freq=TRUE)

#freq4 <- tab(genind2genpop(ndm_pl_gd_new1[loc=c("TCCGCATTTC_CGTTTGAGAC")]),freq=TRUE)

#freq5 <- tab(genind2genpopndm_pl_gd_new1[loc=c("GTCTATCTGA_TCGTCGTCTG")]),freq=TRUE)

par(mfrow=c(2,3), mar=c(5.1,4.1,4.1,.1),las=3)
matplot(freq1, pch=c("a","c"), type="b",
xlab="host",ylab="allele frequency", xaxt="n",
cex=1.5, main="SNP # 1")
axis(side=1, at=1:4, lab=c("Homo Sapien","Pet","Poultry","Swine"))
matplot(freq2, pch=c("c","t"), type="b", xlab="host",
ylab="allele frequency", xaxt="n", cex=1.5,
main="SNP # 2")
axis(side=1, at=1:4, lab=c("Homo Sapien","Pet","Poultry","Swine"))
matplot(freq3, pch=c("c","t"), type="b", xlab="host",
ylab="allele frequency", xaxt="n", cex=1.5,
main="SNP # 3")
axis(side=1, at=1:4, lab=c("Homo Sapien","Pet","Poultry","Swine"))

#summary(ndm_pl_gd_new1$posterior)

#assignplot(dapc.ndm_pl_gd_new1.1)

#compoplot(pnw.dapc,col = function(x) cols, posi = 'top')
compoplot(dapc.ndm_pl_gd_new1.1,col = cols, posi = 'top')

dapc.results <- as.data.frame(dapc.ndm_pl_gd_new1.1$posterior)
dapc.results$pop <- pop(ndm_pl_gd_new1)
dapc.results$indNames <- rownames(dapc.results)

dapc.results <- melt(dapc.results)

colnames(dapc.results) <- c("Original_Pop","Sample","Assigned_Pop","Posterior_membership_probability")

p <- ggplot(dapc.results, aes(x=Sample, y=Posterior_membership_probability, fill=Assigned_Pop))
p <- p + geom_bar(stat='identity') 
p <- p + scale_fill_manual(values = cols) 
p <- p + facet_grid(~Original_Pop, scales = "free")
p <- p + theme(axis.text.x = element_text(angle = 90, hjust = 1, size = 8))
p

ndm_pl_gd_new1.pca <- glPca(ndm_pl_gd_new1, nf = 3)
barplot(100*ndm_pl_gd_new1.pca$eig/sum(ndm_pl_gd_new1.pca$eig), col = heat.colors(50), main="PCA Eigenvalues")
title(ylab="Percent of variance\nexplained", line = 2)
title(xlab="Eigenvalues", line = 1)

ndm_pl_gd_new1.pca

ndm_pl_gd_new1.pca.scores <- as.data.frame(ndm_pl_gd_new1.pca$scores)
ndm_pl_gd_new1.pca.scores$pop <- pop(ndm_pl_gd_new1)

set.seed(9)
p <- ggplot(ndm_pl_gd_new1.pca.scores, aes(x=PC1, y=PC2, colour=pop)) 
p <- p + geom_point(size=2)
p <- p + stat_ellipse(level = 0.95, size = 1)
p <- p + scale_color_manual(values = cols) 
p <- p + geom_hline(yintercept = 0) 
p <- p + geom_vline(xintercept = 0) 
p <- p + theme_bw()

p

### IncX3-Ecoli(TGS——complete genome)(Raw N=98)

In [ ]:
#parsnp/ndmIncX3plasmid_split_files
gene_pa<-read.table('./input/gene_presence_absence.Rtab',header=TRUE)

In [ ]:
row.names(gene_pa) <- gene_pa$Gene
gene_pa <- gene_pa[,-1]
gene_pa <- t(gene_pa)

In [ ]:
gd.ndm98Ecoli <- df2genind(gene_pa,ploidy=2,ncode=1,sep='\t')

In [ ]:
gd.ndm98Ecoli

In [ ]:
indnames<-as.data.frame(indNames(gd.ndm98Ecoli))
colnames(indnames)[1]<-'ACC_NUCCORE'
#indnames['ACC_NUCCORE']<-str_

In [ ]:
incx3.1_host<-subset(incx3.1,select=c('ACC_NUCCORE','host_all'))

In [ ]:
incx3.1_host %>%
dplyr::count(host_all)

In [ ]:
gd.host<-left_join(indnames,incx3.1_host,by='ACC_NUCCORE')

In [ ]:
gd.host[is.na(gd.host$host_all),]$host_all<-'Missing'

In [ ]:
missings<-as.character(gd.host[which(gd.host['host_all']=='Missing'),][['ACC_NUCCORE']])

In [ ]:
pop(gd.ndm98Ecoli)<-gd.host[['host_all']]

In [ ]:
gd.ndm98Ecoli_del <- gd.ndm98Ecoli[!row.names(gd.ndm98Ecoli@tab) %in% missings]

In [ ]:
as.data.frame(pop(gd.ndm98Ecoli_del))%>%
dplyr::count(pop(gd.ndm98Ecoli_del))

In [ ]:
dapc.ndm98Ecoli<- dapc(gd.ndm98Ecoli_del, var.contrib = TRUE, scale = FALSE, n.pca = 30, n.da = nPop(gd.ndm98Ecoli_del) - 1)
#The scatter() function is part of the ade4 package and plots results of a DAPC analysis.
scatter(dapc.ndm98Ecoli, cell = 0, pch = 18:23, cstar = 0, mstree = TRUE, lwd = 2, lty = 2)

In [ ]:
missings2<-as.character(gd.host[which(gd.host['host_all']=='Vegetable'),][['ACC_NUCCORE']])

In [ ]:
missings2

In [ ]:
gd.ndm98Ecoli_del2 <- gd.ndm98Ecoli_del[!row.names(gd.ndm98Ecoli_del@tab) %in% missings2]

In [ ]:
dapc.ndm98Ecoli2<- dapc(gd.ndm98Ecoli_del2, var.contrib = TRUE, scale = FALSE, n.pca = 30, n.da = nPop(gd.ndm98Ecoli_del2) - 1)
#The scatter() function is part of the ade4 package and plots results of a DAPC analysis.
scatter(dapc.ndm98Ecoli2, cell = 0, pch = 18:23, cstar = 0, mstree = TRUE, lwd = 2, lty = 2)

In [ ]:
cols <- brewer.pal(n = 8,name = "Dark2")
cols

In [ ]:
#pdf('./output/fig/ndm98Ecoli2.pdf')
scatter(dapc.ndm98Ecoli2, col = c('#D95F02','#1B9E77','#7570B3'), cex = 2, legend = TRUE, clabel = F, posi.leg = "bottomleft", 
        #scree.pca = TRUE,posi.pca = "topright", 
        posi.da = "topright",
        cleg = 1.2)
#dev.off()

In [ ]:
#tree.ndm98Ecoli2 <- aboot(gd.ndm98Ecoli_del2, tree = "upgma", distance = bitwise.dist, sample = 100, showtree = F, cutoff = 50, quiet = T)

In [ ]:
set.seed(4)
contrib1 <- loadingplot(dapc.ndm98Ecoli2$var.contr, axis = 1, thres = 0.07, lab.jitter = 1)

In [ ]:
freq1 <- tab(genind2genpop(gd.ndm98Ecoli_del2[loc=c("group_85")]),freq=TRUE)

freq2 <- tab(genind2genpop(gd.ndm98Ecoli_del2[loc=c("group_9")]),freq=TRUE)

freq3 <- tab(genind2genpop(gd.ndm98Ecoli_del2[loc=c("group_151")]),freq=TRUE)

par(mfrow=c(1,3), mar=c(5.1,4.1,4.1,.1),las=3)
matplot(freq1, pch=c("a","c"), type="b",
xlab="host",ylab="allele frequency", xaxt="n",
cex=1.5, main="SNP # 1")
axis(side=1, at=1:3, lab=c("Homo Sapien","Animal","Environment"))
matplot(freq2, pch=c("c","t"), type="b", xlab="host",
ylab="allele frequency", xaxt="n", cex=1.5,
main="SNP # 2")
axis(side=1, at=1:3, lab=c("Homo Sapien","Animal","Environment"))
matplot(freq3, pch=c("c","t"), type="b", xlab="host",
ylab="allele frequency", xaxt="n", cex=1.5,
main="SNP # 3")
axis(side=1, at=1:3, lab=c("Homo Sapien","Animal","Environment"))

summary(dapc.ndm98Ecoli2$posterior)

assignplot(dapc.ndm98Ecoli2)

In [ ]:
missings3<-as.character(gd.host[which(gd.host['host_all']=='Environment'),][['ACC_NUCCORE']])

In [ ]:
gd.ndm98Ecoli_del3 <- gd.ndm98Ecoli_del2[!row.names(gd.ndm98Ecoli_del2@tab) %in% missings3]

In [ ]:
dapc.ndm98Ecoli3<- dapc(gd.ndm98Ecoli_del3, var.contrib = TRUE, scale = FALSE, n.pca = 30, n.da = nPop(gd.ndm98Ecoli_del3) - 1)
#The scatter() function is part of the ade4 package and plots results of a DAPC analysis.
scatter(dapc.ndm98Ecoli3, cell = 0, pch = 18:23, cstar = 0, mstree = TRUE, lwd = 2, lty = 2)

In [ ]:
set.seed(4)
contrib2 <- loadingplot(dapc.ndm98Ecoli3$var.contr, axis = 1, thres = 0.07, lab.jitter = 1)

In [ ]:
freqE <- tab(genind2genpop(gd.ndm98Ecoli_del3[loc=c("group_85")]),freq=TRUE)

In [ ]:
matplot(freqE, pch=c("c","t"), type="b", xlab="host",
ylab="allele frequency", xaxt="n", cex=1.5,
main="SNP # E")
axis(side=1, at=1:3, lab=c("Homo Sapien","Animal","Environment"))

### IncX3-K(TGS+NGS——Conserved region)(Raw N=)

## ST tanglegram

### IncX3-Ecoli

In [ ]:
set.seed(1131)
col_tan<-sample(color,207)

In [ ]:
#e
tangle_dir='/root/jupyter_projects/ndm_plasmid_2021/input/tangle/'
strain_tree='/root/jupyter_projects/ndm_plasmid_2021/input/tangle/e_strain.tree'

In [ ]:
#plasmid_tree='/root/jupyter_projects/ndm_plasmid_2021/input/tangle/e_plasmid_incfia.tree'
tree2<-tree_hc_mash

In [ ]:
tree1 <- ape::read.tree(file = strain_tree)
tree1 <- phytools::midpoint.root(tree1)

tree1 <- compute.brlen(tree1)

In [ ]:
#tree2<- ape::read.tree(file = plasmid_tree)                                                                                                                                                                  tree2 <- ape::read.tree(file = plasmid_tree)tree2 <- phytools::midpoint.root(tree2)
tree2 <- compute.brlen(tree2)

In [ ]:
dend1<- as.dendrogram(tree1)
dend2<- as.dendrogram(tree2)

dend1 <- set(   dend1,   what = c( "labels_to_character"), ) 
dend2 <- set(   dend2,   what = c( "labels_to_character"), ) 

In [ ]:
#e
e_mlst<-read.csv(paste0(tangle_dir,'e_all.MLST.tab'),header=FALSE,sep='\t')


labels_dend1 <- as.data.frame(labels(dend1))
colnames(labels_dend1)[1]<-'labels'
colnames(e_mlst)[1]<-'labels'
colnames(e_mlst)[2]<-'ST'

labels_dend1<-left_join(labels_dend1,e_mlst,by='labels')

unique_ST <- data.frame(ST = unique(labels_dend1$ST))

labels_dend1$labels<-as.character(labels_dend1$labels)
#labels_dend1$ST<-as.character(labels_dend1$ST)
head(labels_dend1)
length(unique(labels_dend1$ST))

In [ ]:
#e
set.seed(1128)
#unique_ST$color <- distinctColorPalette(nrow(unique_ST))
unique_ST$color <- sample(col_tan[1:188])

In [ ]:
GCA_ST_color<-left_join(labels_dend1, unique_ST,by='ST')

labels_dend2 <- as.data.frame(labels(dend2))

colnames(labels_dend2)[1]<-'labels'

ST_lab <- GCA_ST_color$ST
ST_lab <- as.factor(ST_lab)
levels(ST_lab) <- 1:length(levels(ST_lab))

GCA_ST_color['factor']<-ST_lab

GCA_ST_color2<-left_join(labels_dend2,GCA_ST_color,by="labels")

plasmid_ST <- GCA_ST_color2 %>%
dplyr::group_by(ST)%>%
dplyr::summarize(Count=n())

plasmid_ST[order(-plasmid_ST$Count),]

legend<-GCA_ST_color2 %>%
dplyr::group_by(ST,color)%>%
dplyr::summarize(Count = n())

head(legend)

In [ ]:
#e
source_info<-unique(subset(predicted_plasmid_contigs,select=c('Assembly.x','Host','Location2')))

head(source_info)

unique_host <- data.frame(Host = unique(source_info$Host))
unique_location <- data.frame(Location2 = unique(source_info$Location2))

nrow(unique_host)

nrow(unique_location)

In [ ]:
#e
unique_host$colors_ho<-col_tan[189:200]



unique_host



#e
unique_location$colors_lo<-col_tan[201:207]



unique_location

source_info<-left_join(source_info,unique_host,by='Host')

source_info<-left_join(source_info,unique_location,by='Location2')

head(source_info)

In [ ]:
dend1 <- dend1 %>% set("labels_colors", "white") %>% set("leaves_cex", 0.15) %>% set("leaves_pch", 19) %>% set("leaves_col", GCA_ST_color[['color']])
dend2 <- dend2 %>% set("labels_colors", "white") %>% set("leaves_cex", 0.15) %>% set("leaves_pch", 19) %>% set("leaves_col", GCA_ST_color2[['color']]) 

In [ ]:
color_line<-left_join(GCA_ST_color,GCA_ST_color2,by='labels')$color.y
color_lines<-color_line[!is.na(color_line)]

lab_info<-source_info
colnames(lab_info)[1]<-'labels'

lab_info[,1]<-gsub('\\.[0-9]','',lab_info[,1])

lab_info<-unique(lab_info)

head(lab_info)

bar1<-left_join(labels_dend1,unique(lab_info),by='labels')

the_bar1<-subset(bar1,select=c('labels','colors_ho','colors_lo'))
#the_bar1$Host<-as.factor(the_bar1$Host)
#the_bar1$Location2<-as.factor(the_bar1$Location2)

labels_dend2 <- as.data.frame(labels(dend2))
colnames(labels_dend2)[1]<-'labels'
labels_dend2$labels<-as.character(labels_dend2$labels)

bar2<-left_join(labels_dend2,unique(bar1),by='labels')

the_bar2<-subset(bar2,select=c('labels','colors_ho','colors_lo'))

#the_bar2$Host<-as.factor(the_bar2$Host)
#the_bar2$Location2<-as.factor(the_bar2$Location2)

rownames(the_bar1)<-the_bar1[,1]
rownames(the_bar2)<-the_bar2[,1]

#bar1_matrix<-data.matrix(the_bar1[,2:3],rownames.force = NA)
#bar2_matrix<-data.matrix(the_bar2[,2:3],rownames.force = NA)

#dim(bar1_matrix)

In [ ]:
par(mar = c(4,4,4,6))
plot(dend2, horiz = TRUE,axes=FALSE)
colored_bars(colors = the_bar2[,3:2], dend = dend2,horiz = TRUE,y_shift = 0.1,sort_by_labels_order = FALSE)
dev.copy(pdf, 'output/e_incx3.1_dend2_complete.pdf', width = 10, height = 11)
dev.off()

#the_bar1_corrected[which(the_bar1_corrected=='GCA_009394145'),]
#the_bar2[which(the_bar2$labels=='GCA_001893245'),]

bar2 %>%
dplyr::group_by(Host) %>%
dplyr::summarize(Count=n())

In [ ]:
inter_list<-intersect_trees(dend1, dend2)

In [ ]:
dend1_corrected<-inter_list[[1]]

labels_dend1_corrected<-as.data.frame(labels(dend1_corrected))
colnames(labels_dend1_corrected)[1]<-'labels'

bar1_corrected<-left_join(labels_dend1_corrected,unique(bar1),by='labels')

the_bar1_corrected<-subset(bar1_corrected,select=c('labels','colors_ho','colors_lo'))

rownames(the_bar1_corrected)<-the_bar1_corrected[,1]

#bar1_corrected_matrix<-data.matrix(the_bar1_corrected[,2:3],rownames.force = NA)

In [ ]:
bar1_corrected %>%
dplyr::group_by(Host) %>%
dplyr::summarize(Count=n())

In [ ]:
par(mar = c(4,4,4,6))
plot(dend1_corrected, horiz = TRUE,axes=FALSE)
colored_bars(colors = the_bar1_corrected[,2:3], dend = dend1_corrected,horiz = TRUE,y_shift = 0.1,sort_by_labels_order = FALSE)
dev.copy(pdf, 'output/e_incx3.1_dend1_corrected_complete.pdf', width = 10, height = 11)
dev.off()

In [ ]:
dendextend::tanglegram(dend1_corrected,dend2, 
                       margin_inner = 5,
                       #lab.cex = 0.3, 
                       lwd = 0.5, edge.lwd = 0.5, type = "r",
                       axes=FALSE,
                       highlight_distinct_edges = FALSE,
                       highlight_branches_lwd = FALSE,
                       color_lines=color_lines,
                       common_subtrees_color_lines = FALSE)
legend("left", legend = legend[['ST']], fill = legend[['color']],cex=0.2)
#colored_bars(colors = bar2_matrix, dend = dend2,horiz = TRUE,y_shift = 0.1)
#colored_bars(colors = bar1_corrected_matrix, dend = dend1_corrected,horiz = TRUE,y_shift = 0.1)
dev.copy(pdf, 'output/e_incx3.1_tangle_complete.pdf', width = 10, height = 11)
dev.off()

In [ ]:
dend12_corrected_1 <- untangle_random_search(dend1_corrected, dend2)
dend12_corrected_2 <- untangle_step_rotate_2side(dend12_corrected_1[[1]], dend12_corrected_1[[2]])

dendextend::tanglegram(dend12_corrected_2[[1]],dend12_corrected_2[[2]],
                       margin_inner = 5,
                       #lab.cex = 0.3, 
                       lwd = 0.5, edge.lwd = 0.5, type = "r",
                       axes=FALSE,
                       highlight_distinct_edges = FALSE,
                       highlight_branches_lwd = FALSE,
                       color_lines=color_lines,
                       common_subtrees_color_lines = FALSE)
legend("left", legend = legend[['ST']], fill = legend[['color']],cex=0.2)
#colored_bars(colors = bar2_matrix, dend = dend2,horiz = TRUE,y_shift = 0.1)
#colored_bars(colors = bar1_corrected_matrix, dend = dend1_corrected,horiz = TRUE,y_shift = 0.1)
dev.copy(pdf, 'output/e_incx3.1_tangle_complete2.pdf', width = 10, height = 11)
dev.off()

#calculate core diff
vcf_file_core='./input/tangle/k_strain.vcf'

ndm_vcf_core <- read.vcfR(vcf_file_core)

ndm_pl_gd_core <- vcfR2genind(ndm_vcf_core)
ndm_pl_gd_core

select<-labels(dend2)

ndm_pl_gd_core1<-ndm_pl_gd_core@ind.names[select]
ndm_pl_gd_core1

ndm_pl_gd_new1 <- ndm_pl_gd[row.names(ndm_pl_gd@tab) %in% select]

ndm_pl_gd_new1

In [ ]:
#calculate SNP diff

In [ ]:
vcf_file_7='./input/tangle/e_plasmid_incfia.vcf'

In [ ]:
ndm_vcf_7 <- read.vcfR(vcf_file_7)
ndm_pl_gd_7 <- vcfR2genind(ndm_vcf_7)
ndm_pl_gd_7

In [ ]:
ndm_pl_gd_ind_7<-as.data.frame(indNames(ndm_pl_gd_7))

colnames(ndm_pl_gd_ind_7)<-'labels'
ndm_pl_pop_7<-left_join(ndm_pl_gd_ind_7,unique(e_mlst),by='labels')

#ndm_pl_pop[which((ndm_pl_pop$Host=='Pet') | (ndm_pl_pop$Host=='Poultry') | (ndm_pl_pop$Host=='Swine')),]$Host<-'Animals'
#ndm_pl_pop[which(!(ndm_pl_pop$Host=='Homo sapiens')),]$Host<-'Others'

In [ ]:
pop(ndm_pl_gd_7)<-ndm_pl_pop_7$ST

pops<-pop(ndm_pl_gd_7)

pops

In [ ]:
myDiff_e <- pairwise_genetic_diff(ndm_vcf_7, pops=pops, method = "nei")

In [ ]:
head(myDiff_e[,1:15])

In [ ]:
colmeans<-colMeans(myDiff_e[,c(4:ncol(myDiff_e))], na.rm = TRUE)

In [ ]:
colmeans<-as.data.frame(colmeans)

In [ ]:
V1<-colnames(colmeans)

In [ ]:
colmeans2<-cbind(V1,colmeans)

In [ ]:
colmeans3<-na.omit(colmeans2[seq(1,nrow(colmeans2),2),])

In [ ]:
head(colmeans3)

In [ ]:
colmeans3[which(colmeans3$colmeans == min(colmeans3$colmeans)),]

In [ ]:
nrow(colmeans3[which(colmeans3$colmeans==1),])

In [ ]:
select2<-as.character(ndm_pl_pop_7[which(ndm_pl_pop_7$ST %in% c('ST101')),][['labels']])

In [ ]:
select2

In [ ]:
adat<-read.table('./input/tangle/e_plasmid_incx3.vcf')

In [ ]:
adat<-adat[,c(10:ncol(adat))]
colnames(adat)<-indNames(ndm_pl_gd_7)

In [ ]:
adat<-adat[,select2]

In [ ]:
nm1 <- outer(colnames(adat), colnames(adat), paste, sep="_")
indx1 <-  which(lower.tri(nm1, diag=TRUE))
res <- outer(1:ncol(adat), 1:ncol(adat), 
              function(x,y) adat[,x]-adat[,y])
colnames(res) <- nm1
res1 <- res[-indx1]
dim(res1)

In [ ]:
colsums<-colSums(abs(res1))

In [ ]:
max(colsums)

### IncX3-K

In [ ]:
#k
tangle_dir='/root/jupyter_projects/ndm_plasmid_2021/input/tangle/'
strain_tree='/root/jupyter_projects/ndm_plasmid_2021/input/tangle/k_strain2.tree'
plasmid_tree='/root/jupyter_projects/ndm_plasmid_2021/input/tangle/k_plasmid_incx3.tree'

In [ ]:
tree1 <- ape::read.tree(file = strain_tree)
tree1 <- phytools::midpoint.root(tree1)

tree1 <- compute.brlen(tree1)

tree2 <- ape::read.tree(file = plasmid_tree)
tree2 <- phytools::midpoint.root(tree2)

tree2 <- compute.brlen(tree2)

dend1<- as.dendrogram(tree1)
dend2<- as.dendrogram(tree2)

dend1 <- set(   dend1,   what = c( "labels_to_character"), ) 
dend2 <- set(   dend2,   what = c( "labels_to_character"), ) 

In [ ]:
#k
k_mlst<-read.csv(paste0(tangle_dir,'k_all.MLST.tab'),header=FALSE,sep='\t')



labels_dend1 <- as.data.frame(labels(dend1))
colnames(labels_dend1)[1]<-'labels'
colnames(k_mlst)[1]<-'labels'
colnames(k_mlst)[2]<-'ST'

labels_dend1<-left_join(labels_dend1,k_mlst,by='labels')

unique_ST <- data.frame(ST = unique(labels_dend1$ST))

labels_dend1$labels<-as.character(labels_dend1$labels)
#labels_dend1$ST<-as.character(labels_dend1$ST)
head(labels_dend1)
length(unique(labels_dend1$ST))

In [ ]:
#k
set.seed(1128)
#unique_ST$color <- distinctColorPalette(nrow(unique_ST))
unique_ST$color <- sample(col_tan[1:108])

In [ ]:
GCA_ST_color<-left_join(labels_dend1, unique_ST,by='ST')

labels_dend2 <- as.data.frame(labels(dend2))

colnames(labels_dend2)[1]<-'labels'

ST_lab <- GCA_ST_color$ST
ST_lab <- as.factor(ST_lab)
levels(ST_lab) <- 1:length(levels(ST_lab))

GCA_ST_color['factor']<-ST_lab

GCA_ST_color2<-left_join(labels_dend2,GCA_ST_color,by="labels")

plasmid_ST <- GCA_ST_color2 %>%
dplyr::group_by(ST)%>%
dplyr::summarize(Count=n())

plasmid_ST[order(-plasmid_ST$Count),]

legend<-GCA_ST_color2 %>%
dplyr::group_by(ST,color)%>%
dplyr::summarize(Count = n())

head(legend)

In [ ]:
#k
source_info<-unique(subset(k_predicted_plasmid_contigs,select=c('Assembly','Host','Location2')))



head(source_info)

unique_host <- data.frame(Host = unique(source_info$Host))
unique_location <- data.frame(Location2 = unique(source_info$Location2))

nrow(unique_host)

nrow(unique_location)

In [ ]:
#k
#unique_host$colors_ho<-cols_ho_lo[1:12]
unique_host$colors_ho<-col_tan[193:200]



unique_host


#k
unique_location$colors_lo<-col_tan[201:206]



unique_location

source_info<-left_join(source_info,unique_host,by='Host')

source_info<-left_join(source_info,unique_location,by='Location2')

head(source_info)

In [ ]:
dend1 <- dend1 %>% set("labels_colors", "white") %>% set("leaves_cex", 0.5) %>% set("leaves_pch", 19) %>% set("leaves_col", GCA_ST_color[['color']])
dend2 <- dend2 %>% set("labels_colors", "white") %>% set("leaves_cex", 0.5) %>% set("leaves_pch", 19) %>% set("leaves_col", GCA_ST_color2[['color']]) 

In [ ]:
color_line<-left_join(GCA_ST_color,GCA_ST_color2,by='labels')$color.y
color_lines<-color_line[!is.na(color_line)]

lab_info<-source_info
colnames(lab_info)[1]<-'labels'

lab_info[,1]<-gsub('\\.[0-9]','',lab_info[,1])

lab_info<-unique(lab_info)

head(lab_info)

In [ ]:
bar1<-left_join(labels_dend1,unique(lab_info),by='labels')

the_bar1<-subset(bar1,select=c('labels','colors_ho','colors_lo'))
#the_bar1$Host<-as.factor(the_bar1$Host)
#the_bar1$Location2<-as.factor(the_bar1$Location2)

In [ ]:
labels_dend2 <- as.data.frame(labels(dend2))
colnames(labels_dend2)[1]<-'labels'
labels_dend2$labels<-as.character(labels_dend2$labels)

bar2<-left_join(labels_dend2,unique(bar1),by='labels')

the_bar2<-subset(bar2,select=c('labels','colors_ho','colors_lo'))

#the_bar2$Host<-as.factor(the_bar2$Host)
#the_bar2$Location2<-as.factor(the_bar2$Location2)

#rownames(the_bar1)<-the_bar1[,1]
#rownames(the_bar2)<-the_bar2[,1]

#bar1_matrix<-data.matrix(the_bar1[,2:3],rownames.force = NA)
#bar2_matrix<-data.matrix(the_bar2[,2:3],rownames.force = NA)

#dim(bar1_matrix)

In [ ]:
par(mar = c(4,4,4,6))
plot(dend2, horiz = TRUE,axes=FALSE)
colored_bars(colors = the_bar2[,3:2], dend = dend2,horiz = TRUE,y_shift = 0.1,sort_by_labels_order = FALSE)
#dev.copy(pdf, 'output/k_incfii_dend2_1.pdf', width = 10, height = 11)
#dev.off()

#the_bar1_corrected[which(the_bar1_corrected=='GCA_009394145'),]
#the_bar2[which(the_bar2$labels=='GCA_001893245'),]

bar2 %>%
dplyr::group_by(Location2) %>%
dplyr::summarize(Count=n())

In [ ]:
inter_list<-intersect_trees(dend1, dend2)

In [ ]:
dend1_corrected<-inter_list[[1]]

labels_dend1_corrected<-as.data.frame(labels(dend1_corrected))
colnames(labels_dend1_corrected)[1]<-'labels'

bar1_corrected<-left_join(labels_dend1_corrected,unique(bar1),by='labels')

the_bar1_corrected<-subset(bar1_corrected,select=c('labels','colors_ho','colors_lo'))

rownames(the_bar1_corrected)<-the_bar1_corrected[,1]

#bar1_corrected_matrix<-data.matrix(the_bar1_corrected[,2:3],rownames.force = NA)

In [ ]:
par(mar = c(4,4,4,6))
plot(dend1_corrected, horiz = TRUE,axes=FALSE)
colored_bars(colors = the_bar1_corrected[,2:3], dend = dend1_corrected,horiz = TRUE,y_shift = 0.1,sort_by_labels_order = FALSE)
#dev.copy(pdf, 'output/k_incfii.1_dend1_corrected.pdf', width = 10, height = 11)
#dev.off()

In [ ]:
dendextend::tanglegram(dend1_corrected,dend2, 
                       margin_inner = 5,
                       #lab.cex = 0.3, 
                       lwd = 0.5, edge.lwd = 0.5, type = "r",
                       axes=FALSE,
                       highlight_distinct_edges = FALSE,
                       highlight_branches_lwd = FALSE,
                       color_lines=color_lines,
                       common_subtrees_color_lines = FALSE)
legend("left", legend = legend[['ST']], fill = legend[['color']],cex=0.4)
#colored_bars(colors = bar2_matrix, dend = dend2,horiz = TRUE,y_shift = 0.1)
#colored_bars(colors = bar1_corrected_matrix, dend = dend1_corrected,horiz = TRUE,y_shift = 0.1)
#dev.copy(pdf, 'output/k_incfii.1_tangle.pdf', width = 10, height = 11)
#dev.off()

In [ ]:
vcf_file_7='./input/tangle/k_plasmid_incx3.vcf'

In [ ]:
ndm_vcf_7 <- read.vcfR(vcf_file_7)
ndm_pl_gd_7 <- vcfR2genind(ndm_vcf_7)
ndm_pl_gd_7

In [ ]:
ndm_pl_gd_ind_7<-as.data.frame(indNames(ndm_pl_gd_7))

colnames(ndm_pl_gd_ind_7)<-'labels'
ndm_pl_pop_7<-left_join(ndm_pl_gd_ind_7,unique(k_mlst),by='labels')

#ndm_pl_pop[which((ndm_pl_pop$Host=='Pet') | (ndm_pl_pop$Host=='Poultry') | (ndm_pl_pop$Host=='Swine')),]$Host<-'Animals'
#ndm_pl_pop[which(!(ndm_pl_pop$Host=='Homo sapiens')),]$Host<-'Others'

In [ ]:
pop(ndm_pl_gd_7)<-ndm_pl_pop_7$ST

pops<-pop(ndm_pl_gd_7)

pops

In [ ]:
myDiff_e <- pairwise_genetic_diff(ndm_vcf_7, pops=pops, method = "nei")

In [ ]:
head(myDiff_e[,1:15])

In [ ]:
colmeans<-colMeans(myDiff_e[,c(4:ncol(myDiff_e))], na.rm = TRUE)

In [ ]:
colmeans<-as.data.frame(colmeans)

In [ ]:
V1<-colnames(colmeans)

In [ ]:
colmeans2<-cbind(V1,colmeans)

In [ ]:
colmeans3<-na.omit(colmeans2[seq(1,nrow(colmeans2),2),])

In [ ]:
head(colmeans3)

In [ ]:
colmeans3[which(colmeans3$colmeans == min(colmeans3$colmeans)),]

In [ ]:
nrow(colmeans3[which(colmeans3$colmeans==1),])

In [ ]:
select2<-as.character(ndm_pl_pop_7[which(ndm_pl_pop_7$ST %in% c('ST11')),][['labels']])

In [ ]:
select2

In [ ]:
adat<-read.table('./input/tangle/k_plasmid_incx3.vcf')

In [ ]:
adat<-adat[,c(10:ncol(adat))]
colnames(adat)<-indNames(ndm_pl_gd_7)

In [ ]:
adat<-adat[,select2]

In [ ]:
nm1 <- outer(colnames(adat), colnames(adat), paste, sep="_")
indx1 <-  which(lower.tri(nm1, diag=TRUE))
res <- outer(1:ncol(adat), 1:ncol(adat), 
              function(x,y) adat[,x]-adat[,y])
colnames(res) <- nm1
res1 <- res[-indx1]
dim(res1)

In [ ]:
colsums<-colSums(abs(res1))

In [ ]:
max(colsums)

# gene context

In [ ]:
library(ggplot2)
library(gggenes)

In [ ]:
CTX <- read.csv("./input/aroundNDM85selected.csv", row.names=NULL)

In [ ]:
CTX$strand <- CTX$strand == "+"

In [ ]:
head(CTX)

In [ ]:
CTX1 <- make_alignment_dummies(
  CTX, aes(xmin = start, xmax = stop, y = molecule, id = gene2),
  on = c("blaNDM"),side = "left")

ggplot(CTX, aes(xmin = start, xmax = end, y = molecule, fill = gene)) +
  geom_gene_arrow() +
  geom_blank(data = CTX1) +
  facet_wrap(~ molecule, scales = "free", ncol = 1) +
  scale_fill_brewer(palette = "Set3") +
  theme_genes()

In [ ]:
colourCount = length(unique(CTX$gene2))

In [ ]:
colourCount

In [ ]:
set.seed(09028)
mycolors<-sample(color,44)

In [ ]:
mycolors

In [ ]:
p <- ggplot(CTX, aes(xmin = start, xmax = stop, y =molecule, fill = gene2,label=gene2,forward=strand)) +
  geom_gene_arrow(arrowhead_height = unit(3, "mm"), arrowhead_width = unit(1, "mm")) +
  geom_gene_label(align = "left") +
  geom_blank(data = CTX1,aes(forward = 1)) +
  facet_wrap(~ molecule, scales = "free", ncol = 1) +
  scale_fill_manual(values=mycolors) +
  theme_genes()


ggsave("./output/09-05-se.pdf",p, limitsize = FALSE,width = 10, height =10 )

# AMR co-network

#########python#########